In [2]:
!pip install streamlit gdown pandas pypdf sentence-transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.3/331.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.0 MB/s eta 0:00:00


Original



In [ ]:
# @title
##Original version
import os
from pathlib import Path
import zipfile
import re

import streamlit as st
import gdown
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# -----------------------------
# STREAMLIT SETUP
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and video recommendations for Class 11–12 students")

# -----------------------------
# DOWNLOAD & EXTRACT NCERT PDFs
# -----------------------------
def download_and_extract():
    if not os.path.exists(ZIP_PATH):
        gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", ZIP_PATH, quiet=False)
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(EXTRACT_DIR)
    # extract nested zips
    for zfile in Path(EXTRACT_DIR).rglob("*.zip"):
        try:
            target = zfile.parent / zfile.stem
            target.mkdir(exist_ok=True)
            with zipfile.ZipFile(zfile, "r") as inner:
                inner.extractall(target)
        except:
            pass

download_and_extract()

# -----------------------------
# PDF Utilities
# -----------------------------
def read_pdf(path):
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""

def clean_text(text):
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()

def load_all_texts():
    texts = []
    paths = []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(pdf)
    return texts, paths

# -----------------------------
# EXTRACT TOPICS
# -----------------------------
def extract_topics_from_text(text):
    lines = text.split("\n")
    topics = []
    for line in lines:
        line = line.strip()
        if 3 <= len(line.split()) <= 8 and line[0].isupper():
            line = re.sub(r"^(chapter\s*\d+:?|[\d.]+\s*)", "", line, flags=re.I)
            topics.append(line.strip())
    return list(set(topics))

@st.cache_data
def get_all_topics():
    texts, _ = load_all_texts()
    all_topics = []
    for text in texts:
        all_topics.extend(extract_topics_from_text(text))
    return sorted(list(set(all_topics)))

all_topics = get_all_topics()

# -----------------------------
# Sentence Transformer
# -----------------------------
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# -----------------------------
# Books & Videos
# -----------------------------
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

videos = {
    subject: {
        level: [f"https://www.youtube.com/watch?v={subject}_{level}_{i+1}" for i in range(3)]
        for level in ["Beginner", "Intermediate", "Advanced"]
    } for subject in SUBJECTS
}

# -----------------------------
# Streamlit UI
# -----------------------------
st.sidebar.header("🎓 Select Options")
selected_subject = st.sidebar.selectbox("Select Subject", SUBJECTS)
selected_level = st.sidebar.selectbox("Select Level", ["Beginner", "Intermediate", "Advanced"])
selected_topics = st.sidebar.multiselect("Select Topic(s)", all_topics)

# -----------------------------
# Recommend chapters using embeddings
# -----------------------------
@st.cache_data
def get_chapter_recommendations(topics, top_n=5):
    if not topics:
        return pd.DataFrame({"Message":["Please select topics"]})

    texts, paths = load_all_texts()
    text_embeddings = embedder.encode(texts, convert_to_tensor=True)
    topic_embedding = embedder.encode([" ".join(topics)], convert_to_tensor=True)

    sims = cosine_similarity(topic_embedding.cpu().numpy(), text_embeddings.cpu().numpy()).flatten()

    top_idx = sims.argsort()[-top_n:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "Chapter PDF": str(paths[i].name),
            "Similarity Score": sims[i]
        })
    return pd.DataFrame(results)

# -----------------------------
# Recommend books & videos
# -----------------------------
def recommend_materials(subject, level, topics):
    if not topics:
        return pd.DataFrame({"Message":["Please select at least one topic"]})
    recommended_books = books.get(subject, {}).get(level, [])
    recommended_videos = videos.get(subject, {}).get(level, [])
    return pd.DataFrame({
        "Topics": [", ".join(topics)],
        "Recommended Books": [", ".join(recommended_books)],
        "Recommended Videos": [", ".join(recommended_videos)]
    })

# -----------------------------
# Display recommendations
# -----------------------------
st.subheader("📚 Chapter Recommendations")
if st.button("Get Chapter Recommendations"):
    chapter_recs = get_chapter_recommendations(selected_topics)
    st.dataframe(chapter_recs)

st.subheader("📖 Book & Video Recommendations")
if st.button("Get Books & Videos"):
    materials = recommend_materials(selected_subject, selected_level, selected_topics)
    st.dataframe(materials)


changed---robust recursive extractor with gdown + requests fallback, ZIP validation, and @st.cache_resource so it only runs once per session.

In [ ]:
# @title
%%writefile app.py
import os
from pathlib import Path
import zipfile
import re

import streamlit as st
import gdown
import requests
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# -----------------------------
# STREAMLIT SETUP
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and video recommendations for Class 11–12 students")

# -----------------------------
# DOWNLOAD & EXTRACT NCERT PDFs
# -----------------------------

def extract_all_zips(folder: str) -> None:
    """Recursively extract all nested ZIPs until none remain unextracted."""
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue                      # already extracted, skip
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True             # new folder found, scan again
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_resource
def download_and_extract():
    # ── 1. DOWNLOAD ──────────────────────────────────────────────────────────
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            # Primary: gdown
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            # Fallback: requests + Drive cookie-confirmation
            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small — likely an error page.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    # ── 2. VALIDATE ──────────────────────────────────────────────────────────
    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP. Check FILE_ID and sharing settings.")
        st.stop()

    # ── 3. EXTRACT OUTER ZIP ─────────────────────────────────────────────────
    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    # ── 4. EXTRACT ALL NESTED ZIPS ───────────────────────────────────────────
    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    # ── 5. COLLECT PDFs ──────────────────────────────────────────────────────
    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files


data_path, pdf_files = download_and_extract()

# -----------------------------
# PDF Utilities
# -----------------------------
def read_pdf(path):
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""

def clean_text(text):
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()

def load_all_texts():
    texts = []
    paths = []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(pdf)
    return texts, paths

# -----------------------------
# EXTRACT TOPICS
# -----------------------------
def extract_topics_from_text(text):
    lines = text.split("\n")
    topics = []
    for line in lines:
        line = line.strip()
        if 3 <= len(line.split()) <= 8 and line[0].isupper():
            line = re.sub(r"^(chapter\s*\d+:?|[\d.]+\s*)", "", line, flags=re.I)
            topics.append(line.strip())
    return list(set(topics))

@st.cache_data
def get_all_topics():
    texts, _ = load_all_texts()
    all_topics = []
    for text in texts:
        all_topics.extend(extract_topics_from_text(text))
    return sorted(list(set(all_topics)))

all_topics = get_all_topics()

# -----------------------------
# Sentence Transformer
# -----------------------------
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# -----------------------------
# Books & Videos
# -----------------------------
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

videos = {
    subject: {
        level: [f"https://www.youtube.com/watch?v={subject}_{level}_{i+1}" for i in range(3)]
        for level in ["Beginner", "Intermediate", "Advanced"]
    } for subject in SUBJECTS
}

# -----------------------------
# Streamlit UI
# -----------------------------
st.sidebar.header("🎓 Select Options")
selected_subject = st.sidebar.selectbox("Select Subject", SUBJECTS)
selected_level = st.sidebar.selectbox("Select Level", ["Beginner", "Intermediate", "Advanced"])
selected_topics = st.sidebar.multiselect("Select Topic(s)", all_topics)

# -----------------------------
# Recommend chapters using embeddings
# -----------------------------
@st.cache_data
def get_chapter_recommendations(topics, top_n=5):
    if not topics:
        return pd.DataFrame({"Message":["Please select topics"]})

    texts, paths = load_all_texts()
    text_embeddings = embedder.encode(texts, convert_to_tensor=True)
    topic_embedding = embedder.encode([" ".join(topics)], convert_to_tensor=True)

    sims = cosine_similarity(topic_embedding.cpu().numpy(), text_embeddings.cpu().numpy()).flatten()

    top_idx = sims.argsort()[-top_n:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "Chapter PDF": str(paths[i].name),
            "Similarity Score": sims[i]
        })
    return pd.DataFrame(results)

# -----------------------------
# Recommend books & videos
# -----------------------------
def recommend_materials(subject, level, topics):
    if not topics:
        return pd.DataFrame({"Message":["Please select at least one topic"]})
    recommended_books = books.get(subject, {}).get(level, [])
    recommended_videos = videos.get(subject, {}).get(level, [])
    return pd.DataFrame({
        "Topics": [", ".join(topics)],
        "Recommended Books": [", ".join(recommended_books)],
        "Recommended Videos": [", ".join(recommended_videos)]
    })

# -----------------------------
# Display recommendations
# -----------------------------
st.subheader("📚 Chapter Recommendations")
if st.button("Get Chapter Recommendations"):
    chapter_recs = get_chapter_recommendations(selected_topics)
    st.dataframe(chapter_recs)

st.subheader("📖 Book & Video Recommendations")
if st.button("Get Books & Videos"):
    materials = recommend_materials(selected_subject, selected_level, selected_topics)
    st.dataframe(materials)


Overwriting app.py


Caching — @st.cache_data on PDF reading, topic extraction, and recommendations; @st.cache_resource on the embedding model. Nothing re-runs unnecessarily.

In [ ]:
# @title
%%writefile app.py
import os
from pathlib import Path
import zipfile
import re

import streamlit as st
import gdown
import requests
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# -----------------------------
# STREAMLIT SETUP
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and video recommendations for Class 11–12 students")

# -----------------------------
# DOWNLOAD & EXTRACT NCERT PDFs
# -----------------------------

def extract_all_zips(folder: str) -> None:
    """Recursively extract all nested ZIPs until none remain unextracted."""
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue                      # already extracted, skip
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True             # new folder found, scan again
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    # ── 1. DOWNLOAD ──────────────────────────────────────────────────────────
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            # Primary: gdown
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            # Fallback: requests + Drive cookie-confirmation
            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small — likely an error page.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    # ── 2. VALIDATE ──────────────────────────────────────────────────────────
    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP. Check FILE_ID and sharing settings.")
        st.stop()

    # ── 3. EXTRACT OUTER ZIP ─────────────────────────────────────────────────
    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    # ── 4. EXTRACT ALL NESTED ZIPS ───────────────────────────────────────────
    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    # ── 5. COLLECT PDFs ──────────────────────────────────────────────────────
    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files


# -----------------------------
# PDF Utilities
# -----------------------------
def read_pdf(path):
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""

def clean_text(text):
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()

@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts = []
    paths = []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))   # str, not Path — serializable by st.cache_data
    return texts, paths

# -----------------------------
# EXTRACT TOPICS
# -----------------------------
def extract_topics_from_text(text):
    # PDF text has no newlines — split on sentence boundaries instead
    segments = re.split(r"[.\n]", text)
    topics = []
    for seg in segments:
        seg = seg.strip()
        if not seg or not seg[0].isupper():          # guard empty / lowercase
            continue
        words = seg.split()
        if 3 <= len(words) <= 8:
            cleaned = re.sub(r"^(chapter\s*\d+:?|[\d.]+\s*)", "", seg, flags=re.I).strip()
            if cleaned:
                topics.append(cleaned)
    return list(set(topics))

@st.cache_data(show_spinner="🔍 Extracting topics…")
def get_all_topics():
    texts, _ = load_all_texts()
    all_topics = []
    for text in texts:
        all_topics.extend(extract_topics_from_text(text))
    return sorted(list(set(all_topics)))

# 1. Ensure data is there
data_path, pdf_files = download_and_extract()

# 2. Only get topics if we have files
if pdf_files:
    all_topics = get_all_topics()
else:
    all_topics = []

# -----------------------------
# Sentence Transformer
# -----------------------------
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# -----------------------------
# Books & Videos
# -----------------------------
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

videos = {
    subject: {
        level: [f"https://www.youtube.com/watch?v={subject}_{level}_{i+1}" for i in range(3)]
        for level in ["Beginner", "Intermediate", "Advanced"]
    } for subject in SUBJECTS
}

# -----------------------------
# Streamlit UI
# -----------------------------
st.sidebar.header("🎓 Select Options")
selected_subject = st.sidebar.selectbox("Select Subject", SUBJECTS)
selected_level = st.sidebar.selectbox("Select Level", ["Beginner", "Intermediate", "Advanced"])
if not all_topics:
    st.sidebar.warning("No topics found yet. Please wait for extraction or check your PDF folder.")
    selected_topics = st.sidebar.multiselect("Select Topic(s)", ["Loading..."], disabled=True)
else:
    selected_topics = st.sidebar.multiselect("Select Topic(s)", all_topics)

# -----------------------------
# Recommend chapters using embeddings
# -----------------------------
@st.cache_data(show_spinner="🔎 Finding relevant chapters…")
def get_chapter_recommendations(topics, top_n=5):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select topics"]})

    texts, paths = load_all_texts()
    text_embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    topic_embedding = embedder.encode([" ".join(topics)], convert_to_numpy=True)

    sims = cosine_similarity(topic_embedding, text_embeddings).flatten()

    top_idx = sims.argsort()[-top_n:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "Chapter PDF": paths[i],
            "Similarity Score": round(float(sims[i]), 4)
        })
    return pd.DataFrame(results)

# -----------------------------
# Recommend books & videos
# -----------------------------
@st.cache_data
def recommend_materials(subject, level, topics):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select at least one topic"]})
    recommended_books = books.get(subject, {}).get(level, [])
    recommended_videos = videos.get(subject, {}).get(level, [])
    return pd.DataFrame({
        "Topics": [", ".join(topics)],
        "Recommended Books": [", ".join(recommended_books)],
        "Recommended Videos": [", ".join(recommended_videos)]
    })

# -----------------------------
# Display recommendations
# -----------------------------
st.subheader("📚 Chapter Recommendations")
if st.button("Get Chapter Recommendations"):
    chapter_recs = get_chapter_recommendations(tuple(selected_topics))
    st.dataframe(chapter_recs)

st.subheader("📖 Book & Video Recommendations")
if st.button("Get Books & Videos"):
    materials = recommend_materials(selected_subject, selected_level, tuple(selected_topics))
    st.dataframe(materials)

Writing app.py


Topics — noisy/empty lines filtered out

In [ ]:
# @title
%%writefile app.py
import os
from pathlib import Path
import zipfile
import re

import streamlit as st
import gdown
import requests
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# -----------------------------
# STREAMLIT SETUP
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and video recommendations for Class 11–12 students")

# -----------------------------
# DOWNLOAD & EXTRACT NCERT PDFs
# -----------------------------

def extract_all_zips(folder: str) -> None:
    """Recursively extract all nested ZIPs until none remain unextracted."""
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue                      # already extracted, skip
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True             # new folder found, scan again
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    # ── 1. DOWNLOAD ──────────────────────────────────────────────────────────
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            # Primary: gdown
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            # Fallback: requests + Drive cookie-confirmation
            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small — likely an error page.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    # ── 2. VALIDATE ──────────────────────────────────────────────────────────
    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP. Check FILE_ID and sharing settings.")
        st.stop()

    # ── 3. EXTRACT OUTER ZIP ─────────────────────────────────────────────────
    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    # ── 4. EXTRACT ALL NESTED ZIPS ───────────────────────────────────────────
    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    # ── 5. COLLECT PDFs ──────────────────────────────────────────────────────
    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files


# -----------------------------
# PDF Utilities
# -----------------------------
def read_pdf(path):
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""

def clean_text(text):
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()

@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts = []
    paths = []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))   # str, not Path — serializable by st.cache_data
    return texts, paths

# -----------------------------
# EXTRACT TOPICS
# -----------------------------
def extract_topics_from_text(text):
    # PDF text has no newlines — split on sentence boundaries instead
    segments = re.split(r"[.\n]", text)
    topics = []
    for seg in segments:
        seg = seg.strip()
        if not seg or not seg[0].isupper():          # guard empty / lowercase
            continue
        words = seg.split()
        if 3 <= len(words) <= 8:
            cleaned = re.sub(r"^(chapter\s*\d+:?|[\d.]+\s*)", "", seg, flags=re.I).strip()
            if cleaned:
                topics.append(cleaned)
    return list(set(topics))

@st.cache_data(show_spinner="🔍 Extracting topics…")
def get_all_topics():
    texts, _ = load_all_texts()
    all_topics = []
    for text in texts:
        all_topics.extend(extract_topics_from_text(text))
    return sorted(list(set(all_topics)))

# 1. Ensure data is there
data_path, pdf_files = download_and_extract()

# 2. Only get topics if we have files
if pdf_files:
    all_topics = get_all_topics()
else:
    all_topics = []

# -----------------------------
# Sentence Transformer
# -----------------------------
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# -----------------------------
# Books & Videos
# -----------------------------
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

videos = {
    subject: {
        level: [f"https://www.youtube.com/watch?v={subject}_{level}_{i+1}" for i in range(3)]
        for level in ["Beginner", "Intermediate", "Advanced"]
    } for subject in SUBJECTS
}

# -----------------------------
# Streamlit UI
# -----------------------------
st.sidebar.header("🎓 Select Options")
selected_subject = st.sidebar.selectbox("Select Subject", SUBJECTS)
selected_level = st.sidebar.selectbox("Select Level", ["Beginner", "Intermediate", "Advanced"])
if not all_topics:
    st.sidebar.warning("No topics found yet. Please wait for extraction or check your PDF folder.")
    selected_topics = st.sidebar.multiselect("Select Topic(s)", ["Loading..."], disabled=True)
else:
    selected_topics = st.sidebar.multiselect("Select Topic(s)", all_topics)

# -----------------------------
# Recommend chapters using embeddings
# -----------------------------
@st.cache_data(show_spinner="🔎 Finding relevant chapters…")
def get_chapter_recommendations(topics, top_n=5):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select topics"]})

    texts, paths = load_all_texts()
    text_embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    topic_embedding = embedder.encode([" ".join(topics)], convert_to_numpy=True)

    sims = cosine_similarity(topic_embedding, text_embeddings).flatten()

    top_idx = sims.argsort()[-top_n:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "Chapter PDF": paths[i],
            "Similarity Score": round(float(sims[i]), 4)
        })
    return pd.DataFrame(results)

# -----------------------------
# Recommend books & videos
# -----------------------------
@st.cache_data
def recommend_materials(subject, level, topics):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select at least one topic"]})
    recommended_books = books.get(subject, {}).get(level, [])
    recommended_videos = videos.get(subject, {}).get(level, [])
    return pd.DataFrame({
        "Topics": [", ".join(topics)],
        "Recommended Books": [", ".join(recommended_books)],
        "Recommended Videos": [", ".join(recommended_videos)]
    })

# -----------------------------
# Display recommendations
# -----------------------------
st.subheader("📚 Chapter Recommendations")
if st.button("Get Chapter Recommendations"):
    chapter_recs = get_chapter_recommendations(tuple(selected_topics))
    st.dataframe(chapter_recs)

st.subheader("📖 Book & Video Recommendations")
if st.button("Get Books & Videos"):
    materials = recommend_materials(selected_subject, selected_level, tuple(selected_topics))
    st.dataframe(materials)

Overwriting app.py


The issue is that NCERT PDFs have very structured text — topics are typically chapter headings, section titles, and key terms that need a smarter extraction strategy. Let me look at what the current extractor actually produces vs. what it should.
The core problems with the current extract_topics_from_text:

Splits on . — breaks mid-sentence phrases like "The Indian Constitution, adopted in 1949, established..." into garbage fragments
3–8 word filter — too broad, picks up random sentences not just headings
No subject-awareness — treats all PDFs the same, mixes Economics fragments with Polity ones

Here's a much smarter extractor:

In [ ]:
# @title
%%writefile app.py
import os
from pathlib import Path
import zipfile
import re

import streamlit as st
import gdown
import requests
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# -----------------------------
# STREAMLIT SETUP
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and video recommendations for Class 11–12 students")

# -----------------------------
# DOWNLOAD & EXTRACT NCERT PDFs
# -----------------------------

def extract_all_zips(folder: str) -> None:
    """Recursively extract all nested ZIPs until none remain unextracted."""
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue                      # already extracted, skip
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True             # new folder found, scan again
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    # ── 1. DOWNLOAD ──────────────────────────────────────────────────────────
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            # Primary: gdown
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            # Fallback: requests + Drive cookie-confirmation
            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small — likely an error page.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    # ── 2. VALIDATE ──────────────────────────────────────────────────────────
    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP. Check FILE_ID and sharing settings.")
        st.stop()

    # ── 3. EXTRACT OUTER ZIP ─────────────────────────────────────────────────
    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    # ── 4. EXTRACT ALL NESTED ZIPS ───────────────────────────────────────────
    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    # ── 5. COLLECT PDFs ──────────────────────────────────────────────────────
    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files


# -----------------------------
# PDF Utilities
# -----------------------------
def read_pdf(path):
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""

def clean_text(text):
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()

@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts = []
    paths = []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))   # str, not Path — serializable by st.cache_data
    return texts, paths

# -----------------------------
# EXTRACT TOPICS
# -----------------------------
# Noise words that indicate a fragment, not a real topic
_NOISE = re.compile(
    r"^(the|a|an|this|that|these|those|it|its|in|on|at|by|for|of|to|and|or|but|"
    r"such|also|thus|hence|therefore|however|moreover|furthermore|although|"
    r"figure|table|box|note|source|see|ref|pg|pp|ibid|op|cit|ncert|reprint)\b",
    re.I
)
_JUNK_CHARS = re.compile(r"[\(\)\[\]\{\}@#$%^&*+=|\\/<>~`\'\"]{2,}")

def extract_topics_from_text(text):
    """
    Extract meaningful topic phrases from PDF text.
    Strategy:
      - Split on newlines AND common sentence-ending punctuation
      - Keep only Title Case or ALL-CAPS short phrases (likely headings)
      - Filter noise words, junk characters, and out-of-range lengths
    """
    # Re-insert newlines at likely heading boundaries (all-caps words or Title Case runs)
    segments = re.split(r"[\n]+|(?<=[a-z])\.\s+(?=[A-Z])", text)
    topics = []
    for seg in segments:
        seg = seg.strip()
        # Basic guards
        if not seg or len(seg) < 5:
            continue
        if _JUNK_CHARS.search(seg):
            continue
        # Strip leading numbering like "1.", "2.3", "Chapter 4"
        cleaned = re.sub(r"^(chapter\s*\d+[:\.]?|unit\s*\d+[:\.]?|[\d]+[\d\.]*\s*)", "", seg, flags=re.I).strip()
        if not cleaned or not cleaned[0].isupper():
            continue
        words = cleaned.split()
        if not (2 <= len(words) <= 7):
            continue
        # Must look like a heading: Title Case or short ALL-CAPS
        is_title_case = sum(1 for w in words if w[0].isupper()) >= len(words) * 0.6
        is_all_caps   = cleaned.isupper() and 2 <= len(words) <= 4
        if not (is_title_case or is_all_caps):
            continue
        # Filter noise starters
        if _NOISE.match(cleaned):
            continue
        # Filter if it ends with a comma or common sentence connectors
        if cleaned.endswith((",", ";", ":", "and", "or", "the")):
            continue
        topics.append(cleaned)
    return list(set(topics))

@st.cache_data(show_spinner="🔍 Extracting topics…")
def get_all_topics():
    texts, _ = load_all_texts()
    all_topics = []
    for text in texts:
        all_topics.extend(extract_topics_from_text(text))
    return sorted(list(set(all_topics)))[:500]

# 1. Ensure data is there
data_path, pdf_files = download_and_extract()

# 2. Only get topics if we have files
if pdf_files:
    all_topics = get_all_topics()
else:
    all_topics = []

# -----------------------------
# Sentence Transformer
# -----------------------------
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# -----------------------------
# Books & Videos
# -----------------------------
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

videos = {
    subject: {
        level: [f"https://www.youtube.com/watch?v={subject}_{level}_{i+1}" for i in range(3)]
        for level in ["Beginner", "Intermediate", "Advanced"]
    } for subject in SUBJECTS
}

# -----------------------------
# Streamlit UI
# -----------------------------
st.sidebar.header("🎓 Select Options")
selected_subject = st.sidebar.selectbox("Select Subject", SUBJECTS)
selected_level = st.sidebar.selectbox("Select Level", ["Beginner", "Intermediate", "Advanced"])
if not all_topics:
    st.sidebar.warning("No topics found yet. Please wait for extraction or check your PDF folder.")
    selected_topics = st.sidebar.multiselect("Select Topic(s)", ["Loading..."], disabled=True)
else:
    selected_topics = st.sidebar.multiselect("Select Topic(s)", all_topics)

# -----------------------------
# Recommend chapters using embeddings
# -----------------------------
@st.cache_data(show_spinner="🔎 Finding relevant chapters…")
def get_chapter_recommendations(topics, top_n=5):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select topics"]})

    texts, paths = load_all_texts()
    text_embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    topic_embedding = embedder.encode([" ".join(topics)], convert_to_numpy=True)

    sims = cosine_similarity(topic_embedding, text_embeddings).flatten()

    top_idx = sims.argsort()[-top_n:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "Chapter PDF": paths[i],
            "Similarity Score": round(float(sims[i]), 4)
        })
    return pd.DataFrame(results)

# -----------------------------
# Recommend books & videos
# -----------------------------
@st.cache_data
def recommend_materials(subject, level, topics):  # topics must be a tuple for cache hashing
    if not topics:
        return pd.DataFrame({"Message":["Please select at least one topic"]})
    recommended_books = books.get(subject, {}).get(level, [])
    recommended_videos = videos.get(subject, {}).get(level, [])
    return pd.DataFrame({
        "Topics": [", ".join(topics)],
        "Recommended Books": [", ".join(recommended_books)],
        "Recommended Videos": [", ".join(recommended_videos)]
    })

# -----------------------------
# Display recommendations
# -----------------------------
st.subheader("📚 Chapter Recommendations")
if st.button("Get Chapter Recommendations"):
    chapter_recs = get_chapter_recommendations(tuple(selected_topics))
    st.dataframe(chapter_recs)

st.subheader("📖 Book & Video Recommendations")
if st.button("Get Books & Videos"):
    materials = recommend_materials(selected_subject, selected_level, tuple(selected_topics))
    st.dataframe(materials)

Writing app.py


ignore below for now


In [ ]:
# @title
%%writefile app.py
import os
import zipfile
import re
from pathlib import Path

import streamlit as st
import gdown
import pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# 1. CONFIG & DATA (Defined First)
# -----------------------------
FILE_ID = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"
SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]

# Recommendation Data
books = {
    "Sociology": {
        "Beginner": ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced": ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"]
    },
    "Polity": {
        "Beginner": ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced": ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"]
    },
    "Psychology": {
        "Beginner": ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Saundra K. Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced": ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"]
    },
    "Economics": {
        "Beginner": ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced": ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"]
    },
    "Business Studies": {
        "Beginner": ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced": ["Advanced Business Management by Robbins", "Strategic Management Advanced"]
    }
}

# -----------------------------
# 2. DOWNLOAD & EXTRACTION
# -----------------------------
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")

def extract_all_zips(folder):
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if file.lower().endswith(".zip"):
                    zip_path = os.path.join(root, file)
                    extract_to = os.path.join(root, os.path.splitext(file)[0])
                    if not os.path.exists(extract_to):
                        try:
                            with zipfile.ZipFile(zip_path, "r") as zf:
                                zf.extractall(extract_to)
                            found_new = True
                        except: pass

@st.cache_resource
def setup_data():
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset..."):
            gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False)

    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting..."):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)
            extract_all_zips(EXTRACT_DIR)

    return [str(p) for p in Path(EXTRACT_DIR).rglob("*.pdf")]

pdf_paths = setup_data()

# -----------------------------
# 3. TEXT PROCESSING
# -----------------------------
@st.cache_data
def process_pdfs(paths):
    texts, valid_paths, topics = [], [], []
    for path in paths:
        try:
            reader = PdfReader(path)
            # Read first few pages for topic extraction and content
            content = " ".join([page.extract_text() or "" for page in reader.pages[:15]])
            if len(content.split()) > 100:
                texts.append(content)
                valid_paths.append(path)
                # Simple topic heuristic: capitalized lines with 3-6 words
                lines = content.split('\n')
                for line in lines[:50]:
                    clean_line = line.strip()
                    if 3 <= len(clean_line.split()) <= 6 and clean_line[0].isupper():
                        topics.append(clean_line)
        except: continue
    return texts, valid_paths, sorted(list(set(topics)))

all_texts, valid_pdf_paths, extracted_topics = process_pdfs(pdf_paths)

# -----------------------------
# 4. EMBEDDINGS & SEARCH
# -----------------------------
@st.cache_resource
def load_model():
    return SentenceTransformer("all-MiniLM-L6-v2")

model = load_model()

@st.cache_data
def get_vectors(_texts):
    return model.encode(_texts, convert_to_tensor=True).cpu().numpy()

# -----------------------------
# 5. UI & OUTPUT
# -----------------------------
if not all_texts:
    st.error("No PDFs found or readable. Please check your data source.")
else:
    vectors = get_vectors(all_texts)

    st.sidebar.header("🎓 Learning Path")
    selected_subject = st.sidebar.selectbox("Subject", SUBJECTS)
    selected_level = st.sidebar.selectbox("Level", ["Beginner", "Intermediate", "Advanced"])

    # Use extracted topics if found, else fallback
    topic_options = extracted_topics if len(extracted_topics) > 5 else ["Constitution", "Demand and Supply", "Social Groups"]
    selected_topics = st.sidebar.multiselect("Select Topics of Interest", topic_options)

    if st.button("🚀 Generate My Learning Plan"):
        if not selected_topics:
            st.warning("Please select at least one topic in the sidebar.")
        else:
            # Chapter Matching
            st.header("📖 Recommended NCERT Chapters")
            query_vec = model.encode([" ".join(selected_topics)])
            scores = cosine_similarity(query_vec, vectors).flatten()
            top_hits = scores.argsort()[-3:][::-1]

            for idx in top_hits:
                p = Path(valid_pdf_paths[idx])
                st.success(f"**{p.name}** — Match Score: {scores[idx]:.2%}")
                st.caption(f"Location: {p.parent.name}")

            st.divider()

            # Resources Matching
            col1, col2 = st.columns(2)
            with col1:
                st.subheader("📚 External Books")
                book_list = books.get(selected_subject, {}).get(selected_level, [])
                for b in book_list:
                    st.write(f"✅ {b}")

            with col2:
                st.subheader("🎥 Video Lectures")
                st.info(f"Searching for {selected_level} content on {selected_subject}...")
                # Dynamic search link
                search_query = "+".join([selected_subject] + selected_topics[:2])
                st.write(f"[Click here to search YouTube for these topics](https://www.youtube.com/results?search_query={search_query})")

Writing app.py


In [ ]:
# @title
%%writefile app.py
import os
import re
import zipfile
from pathlib import Path

import gdown
import pandas as pd
import requests
import streamlit as st
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# CONFIG
# -----------------------------
FILE_ID     = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH    = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"

SUBJECTS = ["Polity", "Economics", "Sociology", "Psychology", "Business Studies"]
LEVELS   = ["Beginner", "Intermediate", "Advanced"]

# ==========================================================
# STREAMLIT PAGE SETUP
# ==========================================================
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")

st.markdown("""
    <style>
        .block-container { padding-top: 2rem; }
        .stButton > button {
            width: 100%;
            background-color: #1a73e8;
            color: white;
            border: none;
            border-radius: 8px;
            padding: 0.5rem 1rem;
            font-weight: 600;
        }
        .stButton > button:hover { background-color: #1558b0; }
        .stDataFrame { border-radius: 8px; }
        h1 { color: #1a73e8; }
        h2, h3 { color: #333; }
    </style>
""", unsafe_allow_html=True)

st.title("📘 NCERT Learning Assistant")
st.caption("Chapter, book, and video recommendations for Class 11–12 students.")

# ==========================================================
# STEP 1 — RECURSIVE ZIP EXTRACTOR
# ==========================================================
def extract_all_zips(folder: str) -> None:
    """
    Recursively find and extract every .zip inside `folder`.
    Keeps looping until no new ZIPs are discovered — handles
    arbitrary nesting depth automatically.
    Warns on corrupt ZIPs and continues; skips already-extracted folders.
    """
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


# ==========================================================
# STEP 2 — DOWNLOAD + EXTRACT  (runs once per session)
# ==========================================================
@st.cache_resource
def download_and_extract(file_id: str = FILE_ID) -> tuple[str, list[str]]:
    """
    1. Download outer ZIP from Google Drive (gdown, then requests fallback).
    2. Validate it is a real ZIP.
    3. Extract outer ZIP into EXTRACT_DIR.
    4. Recursively extract all nested subject/chapter ZIPs.
    5. Return (extract_dir, list_of_all_pdf_paths).
    """

    # --- Download ---
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            downloaded = False
            try:
                gdown.download(id=file_id, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying requests fallback…")

            if not downloaded:
                try:
                    session  = requests.Session()
                    base_url = "https://drive.google.com/uc?export=download"
                    resp     = session.get(base_url, params={"id": file_id}, stream=True)
                    token    = next(
                        (v for k, v in resp.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        resp = session.get(base_url, params={"id": file_id, "confirm": token}, stream=True)
                    else:
                        resp = session.get(
                            f"https://drive.google.com/uc?id={file_id}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in resp.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("File too small — likely an error page, not the ZIP.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    # --- Validate ---
    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP. Check FILE_ID and sharing permissions.")
        st.stop()

    # --- Extract outer ZIP ---
    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    # --- Extract nested subject/chapter ZIPs ---
    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    # --- Collect all PDFs ---
    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files
        if f.lower().endswith(".pdf")
    ]

    st.success(f"✅ Extraction complete — {len(pdf_files)} PDFs ready.")
    return EXTRACT_DIR, pdf_files


data_path, pdf_files = download_and_extract()

# ==========================================================
# PDF UTILITIES
# ==========================================================
def read_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except Exception:
        return ""

def clean_text(text: str) -> str:
    text = re.sub(
        r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*",
        " ", text, flags=re.I,
    )
    return re.sub(r"\s+", " ", text).strip()

@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts() -> tuple[list[str], list[Path]]:
    texts, paths = [], []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(pdf)
    return texts, paths

# ==========================================================
# TOPIC EXTRACTION
# ==========================================================
def extract_topics_from_text(text: str) -> list[str]:
    topics = []
    for line in text.split("\n"):
        line = line.strip()
        if 3 <= len(line.split()) <= 8 and line and line[0].isupper():
            cleaned = re.sub(r"^(chapter\s*\d+:?|[\d.]+\s*)", "", line, flags=re.I).strip()
            if cleaned:
                topics.append(cleaned)
    return list(set(topics))

@st.cache_data(show_spinner="🔍 Extracting topics…")
def get_all_topics() -> list[str]:
    texts, _ = load_all_texts()
    all_topics: set[str] = set()
    for text in texts:
        all_topics.update(extract_topics_from_text(text))
    return sorted(all_topics)[:500]

all_topics = get_all_topics()

# ==========================================================
# SENTENCE TRANSFORMER
# ==========================================================
@st.cache_resource(show_spinner="🤖 Loading embedding model…")
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# ==========================================================
# BOOKS & VIDEOS DATA
# ==========================================================
books = {
    "Sociology": {
        "Beginner":     ["NCERT Sociology Class 11", "Understanding Society by Haralambos (Intro)"],
        "Intermediate": ["Introduction to Sociology by Bottomore", "Sociology: Themes and Perspectives by Haralambos"],
        "Advanced":     ["Advanced Sociology by Giddens", "Sociology: Concepts and Theories by Macionis"],
    },
    "Polity": {
        "Beginner":     ["NCERT Political Science Class 11", "Indian Constitution Made Easy"],
        "Intermediate": ["Indian Polity by M. Laxmikanth", "Governance in India by D.D. Basu"],
        "Advanced":     ["Introduction to the Constitution of India by D.D. Basu", "Indian Constitutional Law Advanced"],
    },
    "Psychology": {
        "Beginner":     ["NCERT Psychology Class 11", "Psychology Basics Intro"],
        "Intermediate": ["Psychology: An Exploration by Ciccarelli", "Understanding Psychology by Feldman"],
        "Advanced":     ["Advanced Psychology by Baron & Misra", "Handbook of Psychology by Weiner"],
    },
    "Economics": {
        "Beginner":     ["NCERT Economics Class 11", "Principles of Economics Intro by Mankiw"],
        "Intermediate": ["Principles of Economics by N. Gregory Mankiw", "Economic Theory by Samuelson"],
        "Advanced":     ["Advanced Economic Theory by H.L. Ahuja", "Micro and Macro Economics Advanced"],
    },
    "Business Studies": {
        "Beginner":     ["NCERT Business Studies Class 11", "Introduction to Management Basics"],
        "Intermediate": ["Business Studies by Poonam Gandhi", "Business Management Concepts"],
        "Advanced":     ["Advanced Business Management by Robbins", "Strategic Management Advanced"],
    },
}

def _yt_search(subject: str, level: str) -> str:
    query = f"NCERT+{subject}+{level}+Class+11+12".replace(" ", "+")
    return f"https://www.youtube.com/results?search_query={query}"

videos = {
    subject: {level: _yt_search(subject, level) for level in LEVELS}
    for subject in SUBJECTS
}

# ==========================================================
# CHAPTER RECOMMENDATIONS
# ==========================================================
@st.cache_data(show_spinner="🔎 Computing recommendations…")
def get_chapter_recommendations(topics: tuple, top_n: int = 5) -> pd.DataFrame:
    if not topics:
        return pd.DataFrame({"Message": ["Please select at least one topic."]})

    texts, paths = load_all_texts()
    if not texts:
        return pd.DataFrame({"Message": ["No PDF text found — check extraction."]})

    text_embs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    topic_emb = embedder.encode([" ".join(topics)], convert_to_numpy=True)
    sims      = cosine_similarity(topic_emb, text_embs).flatten()
    top_idx   = sims.argsort()[-top_n:][::-1]

    return pd.DataFrame([
        {"Chapter PDF": paths[i].name, "Similarity Score": round(float(sims[i]), 4)}
        for i in top_idx
    ])

# ==========================================================
# BOOK & VIDEO RECOMMENDATIONS
# ==========================================================
def recommend_materials(subject: str, level: str, topics: list[str]) -> pd.DataFrame:
    if not topics:
        return pd.DataFrame({"Message": ["Please select at least one topic."]})
    return pd.DataFrame({
        "Subject":             [subject],
        "Level":               [level],
        "Topics":              [", ".join(topics)],
        "Recommended Books":   [" | ".join(books.get(subject, {}).get(level, []))],
        "YouTube Search Link": [videos.get(subject, {}).get(level, "")],
    })

# ==========================================================
# SIDEBAR
# ==========================================================
st.sidebar.header("🎓 Select Options")

col_m1, col_m2, col_m3 = st.sidebar.columns(3)
col_m1.metric("PDFs", len(pdf_files))
col_m2.metric("Topics", len(all_topics))
col_m3.metric("Subjects", len(SUBJECTS))

st.sidebar.divider()

selected_subject = st.sidebar.selectbox("Subject", SUBJECTS)
selected_level   = st.sidebar.selectbox("Level",   LEVELS)
selected_topics  = st.sidebar.multiselect(
    "Topic(s)",
    all_topics,
    help="Select one or more topics to get personalized recommendations."
)

# ==========================================================
# MAIN LAYOUT
# ==========================================================
col1, col2 = st.columns(2, gap="large")

with col1:
    st.subheader("📚 Chapter Recommendations")
    st.caption("Most relevant chapters from the NCERT PDFs based on your topics.")
    if st.button("Get Chapter Recommendations", key="ch_btn"):
        if not selected_topics:
            st.warning("Please select at least one topic from the sidebar.")
        else:
            df = get_chapter_recommendations(tuple(selected_topics))
            st.dataframe(df, use_container_width=True, hide_index=True)

with col2:
    st.subheader("📖 Books & Videos")
    st.caption("Curated books and YouTube resources for your subject and level.")
    if st.button("Get Books & Videos", key="bv_btn"):
        if not selected_topics:
            st.warning("Please select at least one topic from the sidebar.")
        else:
            df = recommend_materials(selected_subject, selected_level, selected_topics)
            st.dataframe(df, use_container_width=True, hide_index=True)
            yt_link = videos.get(selected_subject, {}).get(selected_level, "")
            if yt_link:
                st.markdown(f"🔗 [Search YouTube → {selected_subject} ({selected_level})]({yt_link})")

Overwriting app.py


database added

In [10]:
# @title
%%writefile app.py
import os
import re
import uuid
import zipfile
from pathlib import Path

import streamlit as st
import gdown
import requests
import pandas as pd
import sqlite3
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
FILE_ID     = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH    = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"
DB_PATH     = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"

# ─────────────────────────────────────────────
# PAGE CONFIG
# ─────────────────────────────────────────────
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and research paper recommendations for Class 11–12 students")

# ─────────────────────────────────────────────
# MOUNT GOOGLE DRIVE
# ─────────────────────────────────────────────
try:
    from google.colab import drive
    if not os.path.exists("/content/drive/MyDrive"):
        drive.mount("/content/drive")
except ImportError:
    DB_PATH = "ncert.db"   # fallback for local runs

# ─────────────────────────────────────────────
# DATABASE — connect only, no table creation
# ─────────────────────────────────────────────
@st.cache_resource
def get_db():
    if not os.path.exists(DB_PATH):
        st.error(f"❌ Database not found at:\n`{DB_PATH}`\n\nPlease run setup_db.ipynb first, or update DB_PATH at the top of app.py.")
        st.stop()
    try:
        conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        conn.row_factory = sqlite3.Row
        # Quick sanity check — if file is corrupt/not a DB this will fail fast
        conn.execute("SELECT 1")
        return conn
    except sqlite3.DatabaseError as e:
        st.error(f"❌ File exists but is not a valid SQLite database:\n`{DB_PATH}`\n\nError: {e}\n\nPlease re-run setup_db.ipynb to recreate it.")
        st.stop()


def get_subjects() -> list:
    """Load all subjects from DB — adding a new subject to DB instantly appears in app."""
    cur = get_db().cursor()
    cur.execute("SELECT subject_name FROM subjects ORDER BY subject_name")
    return [row["subject_name"] for row in cur.fetchall()]


def get_levels(subject_name: str = None) -> list:
    """
    Load distinct levels available for a subject from recommendations table.
    Order: Beginner → Intermediate → Advanced.
    Adding a new level to DB instantly appears in app.
    """
    cur = get_db().cursor()
    level_order = "CASE level WHEN 'Beginner' THEN 1 WHEN 'Intermediate' THEN 2 WHEN 'Advanced' THEN 3 ELSE 4 END"
    if subject_name:
        subject_id = subject_name.lower().replace(" ", "_")
        cur.execute(f"""
            SELECT DISTINCT level FROM recommendations
            WHERE subject_id = ?
            ORDER BY {level_order}
        """, (subject_id,))
    else:
        cur.execute(f"""
            SELECT DISTINCT level FROM recommendations
            ORDER BY {level_order}
        """)
    return [row["level"] for row in cur.fetchall()]


def get_recommendations(subject_name: str, level: str) -> list:
    subject_id = subject_name.lower().replace(" ", "_")
    cur = get_db().cursor()
    cur.execute("""
        SELECT title, author, level_type, why, link, journal, ncert_chapter_link
        FROM recommendations
        WHERE subject_id = ? AND level = ?
        ORDER BY rec_id
    """, (subject_id, level))
    return [dict(row) for row in cur.fetchall()]


def save_topics_to_db(topics: list, source_pdf: str) -> None:
    conn = get_db()
    conn.executemany(
        "INSERT OR IGNORE INTO topics (topic_text, source_pdf) VALUES (?, ?)",
        [(t, source_pdf) for t in topics]
    )
    conn.commit()


def load_topics_from_db() -> list:
    cur = get_db().cursor()
    cur.execute("SELECT DISTINCT topic_text FROM topics ORDER BY topic_text LIMIT 500")
    return [row["topic_text"] for row in cur.fetchall()]


def load_topics_from_chapters(subject_id: str = None) -> list:
    """
    Load clean topic list from the chapters table.
    These are real NCERT chapter names — no noise.
    Optionally filter by subject.
    """
    cur = get_db().cursor()
    if subject_id:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0 AND subject_id = ?
            ORDER BY chapter_name
        """, (subject_id,))
    else:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0
            ORDER BY chapter_name
        """)
    return [row["chapter_name"] for row in cur.fetchall()]


'''def log_user_search(session_id: str, subject: str, level: str, topics: list) -> None:
    conn = get_db()
    conn.execute("""
        INSERT INTO user_history (session_id, subject_id, level, selected_topics)
        VALUES (?, ?, ?, ?)
    """, (session_id, subject.lower().replace(" ", "_"), level, str(topics)))
    conn.commit()'''

# ─────────────────────────────────────────────
# DOWNLOAD & EXTRACT NCERT PDFs
# ─────────────────────────────────────────────
def extract_all_zips(folder: str) -> None:
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP.")
        st.stop()

    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files

# ─────────────────────────────────────────────
# PDF UTILITIES
# ─────────────────────────────────────────────
def read_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""


def clean_text(text: str) -> str:
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()


@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts, paths = [], []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))
    return texts, paths

# ─────────────────────────────────────────────
# TOPICS — extracted from PDFs, cached in DB
# ─────────────────────────────────────────────

# ── Noise patterns to reject ──────────────────
# Bibliography entries: "SHARMA, R. 2005. Society..."
_AUTHOR_ENTRY   = re.compile(r"^[A-Z]{2,},\s+[A-Z]")
# Any line with a year like "1970." or "(2005)"
_HAS_YEAR       = re.compile(r"(19|20)\d{2}")
# Publisher / place names
_PUBLISHER      = re.compile(
    r"(press|publishing|publishers|edition|reprint|oxford|cambridge|mcgraw|"
    r"pearson|routledge|sage|wiley|elsevier|springer|penguin|harper|norton|"
    r"new york|new delhi|london|chicago|boston|mumbai|kolkata|chennai|"
    r"pp\.|vol\.|ibid|op\.cit|et al|isbn|doi|http|www\.)",
    re.I
)
# Lines that start with noise words
_NOISE_START    = re.compile(
    r"^(the|a|an|this|that|these|it|its|in|on|at|by|for|of|to|and|or|but|"
    r"such|also|thus|hence|therefore|however|moreover|furthermore|although|"
    r"figure|table|box|note|source|see|pg|pp|ref|ncert|reprint|let us|"
    r"activity|exercise|project|chapter|unit|section)",
    re.I
)
# Must look like a proper heading: starts uppercase, only letters/spaces/hyphens
_VALID_HEADING  = re.compile(r"^[A-Z][a-zA-Z\s\-:']{3,55}$")


def extract_topics_from_text(text: str) -> list:
    """
    Strict extractor — only picks real section/chapter headings from PDF text.

    Rules:
    - 2 to 6 words only
    - Must start with uppercase
    - Must be mostly title-cased (≥70% words capitalised)
    - No years, no author surname patterns, no publisher names
    - No punctuation endings, no digits
    - Rejects known noise start words
    """
    lines = text.split("\n")
    topics = set()

    for raw_line in lines:
        line = raw_line.strip()

        # ── Basic length filter ────────────────
        words = line.split()
        if not (2 <= len(words) <= 6):
            continue

        # ── Hard rejections ────────────────────
        if _AUTHOR_ENTRY.match(line):      # "SHARMA, R. 2005..."
            continue
        if _HAS_YEAR.search(line):         # anything with 1970, 2005 etc
            continue
        if _PUBLISHER.search(line):        # publisher / place names
            continue
        if _NOISE_START.match(line):       # starts with filler word
            continue
        if not _VALID_HEADING.match(line): # must match heading pattern
            continue

        # ── Endings filter ─────────────────────
        if line[-1] in ".,;:":
            continue
        last_word = words[-1].lower()
        if last_word in ("and", "or", "the", "of", "a", "an", "in", "on"):
            continue

        # ── Digits filter ─────────────────────
        if any(c.isdigit() for c in line):
            continue

        # ── Title-case check ──────────────────
        # At least 70% of words must start with uppercase
        titled = sum(1 for w in words if w and w[0].isupper())
        if titled / len(words) < 0.70:
            continue

        topics.add(line)

    return list(topics)


def topics_in_db() -> bool:
    """Quick check — are any topics already saved in the DB?"""
    cur = get_db().cursor()
    cur.execute("SELECT COUNT(*) as n FROM topics")
    return cur.fetchone()["n"] > 0


@st.cache_data(show_spinner=False)
def get_all_topics(subject_name: str = None) -> list:
    """
    Fast path: DB already has topics → pure SQL query, instant.
    Slow path: DB empty → extract from PDFs once, save, never slow again.
    """
    cur = get_db().cursor()

    # ── Fast path: topics already in DB ───────────────────────────────────
    if topics_in_db():
        if subject_name:
            subject_id = subject_name.lower().replace(" ", "_")
            cur.execute("""
                SELECT DISTINCT t.topic_text FROM topics t
                WHERE t.source_pdf IN (
                    SELECT DISTINCT pdf_filename FROM chapters
                    WHERE subject_id = ?
                )
                ORDER BY t.topic_text
            """, (subject_id,))
        else:
            cur.execute("""
                SELECT DISTINCT topic_text FROM topics
                ORDER BY topic_text LIMIT 600
            """)
        return [r["topic_text"] for r in cur.fetchall()]

    # ── Slow path: extract from PDFs (first time only) ────────────────────
    with st.spinner("🔍 Extracting topics from PDFs — first time only, won't repeat..."):
        texts, paths = load_all_texts()
        for text, path in zip(texts, paths):
            extracted = extract_topics_from_text(text)
            save_topics_to_db(extracted, path)

    # Now re-query with the fresh data
    return get_all_topics(subject_name)


# ─────────────────────────────────────────────
# EMBEDDER
# ─────────────────────────────────────────────
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# ─────────────────────────────────────────────
# BOOT SEQUENCE
# ─────────────────────────────────────────────
_, pdf_files = download_and_extract()
# PDF embeddings and topics load lazily:
#   - topics: loaded when sidebar dropdown opens (DB query, instant if cached)
#   - embeddings: loaded when first search runs (cached after that)

# Session ID for anonymous history logging
'''if "session_id" not in st.session_state:
    st.session_state["session_id"] = str(uuid.uuid4())'''

# ─────────────────────────────────────────────
# SIDEBAR
# ─────────────────────────────────────────────
st.sidebar.header("🎓 Select Options")
# Loaded from DB — adding a new subject/level in DB instantly shows here
all_subjects     = get_subjects()
selected_subject = st.sidebar.selectbox("Select Subject", all_subjects)
all_levels       = get_levels(selected_subject)
selected_level   = st.sidebar.selectbox("Select Level", all_levels)

st.sidebar.markdown("---")

# ── Input mode toggle ──────────────────────────
input_mode = st.sidebar.radio(
    "Search by",
    ["📝 Type a query", "📋 Pick from topics"],
    help="Type your own question OR pick chapter names from the list"
)

user_query = ""
selected_topics = []

if input_mode == "📝 Type a query":
    user_query = st.sidebar.text_area(
        "Your question or keyword",
        placeholder="e.g. what is the caste system?\nor: federalism in India\nor: GDP and national income",
        height=100,
        help="Type anything — a question, keyword, or concept"
    )
    # derive topics list from query for YouTube/books (use query words)
    selected_topics = [w for w in user_query.split() if len(w) > 3] if user_query else []

else:
    subject_topics = get_all_topics(selected_subject)
    if not subject_topics:
        st.sidebar.warning("No topics found — PDFs will be scanned on first search.")
        selected_topics = []
    else:
        st.sidebar.caption(f"{len(subject_topics)} topics from {selected_subject} PDFs")
        selected_topics = st.sidebar.multiselect(
            "Select Topic(s)",
            subject_topics,
            help="Topics extracted from NCERT PDF text"
        )
        # Reset button — wipes DB cache and re-extracts from PDFs
        if st.sidebar.button("🔄 Reset & Re-extract Topics", help="Use if topics look noisy"):
            conn = get_db()
            conn.execute("DELETE FROM topics")
            conn.commit()
            st.cache_data.clear()
            st.rerun()
    # join selected topics into a query string for the embedder
    user_query = " ".join(selected_topics)

# Final search query used everywhere
search_query = user_query.strip()

# ─────────────────────────────────────────────
# CHAPTER RECOMMENDATIONS (embeddings)
# ─────────────────────────────────────────────
def lookup_chapter(pdf_filename: str, zip_source: str = None) -> dict:
    """
    Given a pdf_filename (and optionally zip_source for disambiguation),
    return readable chapter details from the DB.
    zip_source matters for Business Studies — Class 11 & 12 share identical filenames.
    Falls back to raw filename if not found.
    """
    cur = get_db().cursor()
    if zip_source:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name, book_name, is_special
            FROM chapters WHERE pdf_filename = ? AND zip_source = ?
        """, (pdf_filename, zip_source))
    else:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name, book_name, is_special
            FROM chapters WHERE pdf_filename = ? LIMIT 1
        """, (pdf_filename,))
    row = cur.fetchone()
    if row:
        return dict(row)
    return {
        "subject_id": "—", "class": "—", "chapter_number": "—",
        "chapter_name": pdf_filename, "book_name": "—", "is_special": 0
    }


@st.cache_resource(show_spinner="📐 Indexing PDF chapters (one-time)…")
def get_pdf_embeddings():
    """
    Encode ALL PDF texts ONCE and cache in memory.
    Any query then just encodes a single string and does cosine similarity — instant.
    """
    texts, paths = load_all_texts()
    embs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    return texts, paths, embs


def get_chapter_recommendations(query: str, top_n=5):
    """
    Search chapters using free text OR chapter name keywords.
    e.g. "what is the caste system", "GDP national income", "Federalism"
    PDF embeddings are pre-cached so each search is instant.
    """
    if not query or not query.strip():
        return pd.DataFrame()

    texts, paths, text_emb = get_pdf_embeddings()   # cached — no recompute
    topic_emb = embedder.encode([query.strip()], convert_to_numpy=True)
    sims      = cosine_similarity(topic_emb, text_emb).flatten()
    top_idx   = sims.argsort()[-top_n:][::-1]

    rows = []
    for i in top_idx:
        info = lookup_chapter(paths[i])
        if info.get("is_special"):
            continue
        rows.append({
            "Subject":      info["subject_id"].replace("_", " ").title(),
            "Class":        f"Class {info['class']}",
            "Book":         info["book_name"],
            "Ch No.":       info["chapter_number"],
            "Chapter Name": info["chapter_name"],
            "Relevance":    round(float(sims[i]), 4),
        })
    return pd.DataFrame(rows) if rows else pd.DataFrame()

# ─────────────────────────────────────────────
# BOOK & PAPER RECOMMENDATIONS (from DB)
# ─────────────────────────────────────────────
def recommend_materials(subject: str, level: str, topics: tuple) -> pd.DataFrame:
    if not topics:
        return pd.DataFrame({"Message": ["Please select at least one topic"]})
    rows = get_recommendations(subject, level)
    if not rows:
        return pd.DataFrame({"Message": [f"No data found for {subject} / {level}"]})
    return pd.DataFrame([{
        "Type":          r["level_type"],
        "Title":         r["title"],
        "Author":        r["author"],
        "Why Read This": r["why"],
        "NCERT Chapter": r["ncert_chapter_link"] or "—",
        "Link":          r["link"],
    } for r in rows])

# ─────────────────────────────────────────────
# YOUTUBE SEARCH URL BUILDER
# ─────────────────────────────────────────────
def build_youtube_url(subject: str, level: str, topics: list) -> str:
    """
    Build a YouTube search URL from subject + level + selected topics.
    Opens YouTube search results directly — no API key needed.
    """
    import urllib.parse
    # Compose a focused search query
    topic_str = " ".join(topics[:3]) if topics else ""
    query = f"NCERT {subject} {level} {topic_str} class 11 12".strip()
    encoded = urllib.parse.quote_plus(query)
    return f"https://www.youtube.com/results?search_query={encoded}"


# ─────────────────────────────────────────────
# RENDER — Tabs layout
# ─────────────────────────────────────────────
st.markdown("---")

tab1, tab2, tab3 = st.tabs([
    "📄 Chapter Recommendations",
    "📚 Books & Research Papers",
    "▶️ YouTube Videos",
])

# ── Tab 1: Chapter Recommendations ────────────
with tab1:
    st.markdown(
        "Finds the most relevant NCERT chapters based on your query. "
        "Works with free text — *what is caste system*, *explain GDP*, anything."
    )
    if not search_query:
        st.info("👈 Type a question or select chapters from the sidebar to search.")
    else:
        st.markdown(f"🔎 **Searching for:** `{search_query}`")
        # Auto-search on every new query — no button needed
        with st.spinner("Finding relevant chapters..."):
            df = get_chapter_recommendations(search_query)
        if df.empty:
            st.warning("No matching chapters found. Try different keywords.")
        else:
            st.success(f"✅ Top {len(df)} chapters found!")
            st.dataframe(
                df,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Relevance": st.column_config.ProgressColumn(
                        "Relevance",
                        min_value=0,
                        max_value=1,
                        format="%.4f"
                    )
                }
            )

# ── Tab 2: Books & Research Papers ────────────
with tab2:
    st.markdown(
        f"Showing **{selected_level}** level resources for **{selected_subject}** "
        f"— NCERT textbooks → reference books → research papers."
    )
    if st.button("📖 Get Books & Papers", type="primary", key="book_btn"):
        '''log_user_search(
            st.session_state["session_id"],
            selected_subject,
            selected_level,
            selected_topics if selected_topics else [search_query]
        )'''
        rows = get_recommendations(selected_subject, selected_level)
        if not rows:
            st.warning(f"No recommendations found for {selected_subject} / {selected_level}")
        else:
            for r in rows:
                type_icons = {"NCERT": "📗", "Book": "📘", "Paper": "📄"}
                icon = type_icons.get(r["level_type"], "📌")
                with st.expander(f"{icon} {r['title']} — *{r['author']}*", expanded=False):
                    st.markdown(f"**Type:** {r['level_type']}")
                    if r["journal"]:
                        st.markdown(f"**Journal:** {r['journal']}")
                    if r["ncert_chapter_link"]:
                        st.markdown(f"**NCERT Chapter:** `{r['ncert_chapter_link']}`")
                    st.markdown(f"💡 *{r['why']}*")
                    if r["link"]:
                        st.markdown(f"[🔗 Open Resource]({r['link']})")

# ── Tab 3: YouTube Videos ──────────────────────
with tab3:
    st.markdown(
        "Clicking the button below will open a **YouTube search** in a new tab "
        "with a query built from your selected subject, level, and topics."
    )

    # Show the query that will be used
    yt_query_preview = f"NCERT {selected_subject} {selected_level} {search_query} class 11 12".strip()
    st.markdown(f"**Search query:** `{yt_query_preview}`")

    if not search_query:
        st.info("👈 Type a query or select topics to refine the YouTube search.")

    # Always show the button — even without topics it gives subject+level results
    yt_url = build_youtube_url(selected_subject, selected_level, [search_query])
    st.link_button("▶️ Search on YouTube", yt_url, type="primary")

    st.markdown("---")
    # Also show quick-access searches for each level
    st.markdown("**Or jump directly to a level:**")
    level_icons = {"Beginner": "🌱", "Intermediate": "📚", "Advanced": "🔬"}
    yt_levels = get_levels(selected_subject)
    cols = st.columns(len(yt_levels))
    for col, lvl in zip(cols, yt_levels):
        with col:
            icon = level_icons.get(lvl, "▶️")
            url  = build_youtube_url(selected_subject, lvl, [search_query])
            st.link_button(f"{icon} {lvl}", url, use_container_width=True)

'''# ─────────────────────────────────────────────
# SIDEBAR — search history
# ─────────────────────────────────────────────
with st.sidebar.expander("🗂️ View Search History"):
    cur = get_db().cursor()
    cur.execute("""
        SELECT subject_id, level, selected_topics, searched_at
        FROM user_history ORDER BY searched_at DESC LIMIT 20
    """)
    history = [dict(r) for r in cur.fetchall()]
    if history:
        st.dataframe(pd.DataFrame(history), use_container_width=True)
    else:
        st.info("No searches logged yet.")'''

Overwriting app.py


User History Removed

In [12]:
# @title
%%writefile app.py
import os
import re
import uuid
import zipfile
from pathlib import Path

import streamlit as st
import gdown
import requests
import pandas as pd
import sqlite3
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
FILE_ID     = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH    = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"
DB_PATH     = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"

# ─────────────────────────────────────────────
# PAGE CONFIG
# ─────────────────────────────────────────────
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and research paper recommendations for Class 11–12 students")

# ─────────────────────────────────────────────
# MOUNT GOOGLE DRIVE
# ─────────────────────────────────────────────
try:
    from google.colab import drive
    if not os.path.exists("/content/drive/MyDrive"):
        drive.mount("/content/drive")
except ImportError:
    DB_PATH = "ncert.db"   # fallback for local runs

# ─────────────────────────────────────────────
# DATABASE — connect only, no table creation
# ─────────────────────────────────────────────
@st.cache_resource
def get_db():
    if not os.path.exists(DB_PATH):
        st.error(f"❌ Database not found at:\n`{DB_PATH}`\n\nPlease run setup_db.ipynb first, or update DB_PATH at the top of app.py.")
        st.stop()
    try:
        conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        conn.row_factory = sqlite3.Row
        # Quick sanity check — if file is corrupt/not a DB this will fail fast
        conn.execute("SELECT 1")
        return conn
    except sqlite3.DatabaseError as e:
        st.error(f"❌ File exists but is not a valid SQLite database:\n`{DB_PATH}`\n\nError: {e}\n\nPlease re-run setup_db.ipynb to recreate it.")
        st.stop()


def get_subjects() -> list:
    """Load all subjects from DB — adding a new subject to DB instantly appears in app."""
    cur = get_db().cursor()
    cur.execute("SELECT subject_name FROM subjects ORDER BY subject_name")
    return [row["subject_name"] for row in cur.fetchall()]


def get_levels(subject_name: str = None) -> list:
    """
    Load distinct levels available for a subject from recommendations table.
    Order: Beginner → Intermediate → Advanced.
    Adding a new level to DB instantly appears in app.
    """
    cur = get_db().cursor()
    level_order = "CASE level WHEN 'Beginner' THEN 1 WHEN 'Intermediate' THEN 2 WHEN 'Advanced' THEN 3 ELSE 4 END"
    if subject_name:
        subject_id = subject_name.lower().replace(" ", "_")
        cur.execute(f"""
            SELECT DISTINCT level FROM recommendations
            WHERE subject_id = ?
            ORDER BY {level_order}
        """, (subject_id,))
    else:
        cur.execute(f"""
            SELECT DISTINCT level FROM recommendations
            ORDER BY {level_order}
        """)
    return [row["level"] for row in cur.fetchall()]


def get_recommendations(subject_name: str, level: str) -> list:
    subject_id = subject_name.lower().replace(" ", "_")
    cur = get_db().cursor()
    cur.execute("""
        SELECT title, author, level_type, why, link, journal, ncert_chapter_link
        FROM recommendations
        WHERE subject_id = ? AND level = ?
        ORDER BY rec_id
    """, (subject_id, level))
    return [dict(row) for row in cur.fetchall()]


def save_topics_to_db(topics: list, source_pdf: str) -> None:
    conn = get_db()
    conn.executemany(
        "INSERT OR IGNORE INTO topics (topic_text, source_pdf) VALUES (?, ?)",
        [(t, source_pdf) for t in topics]
    )
    conn.commit()


def load_topics_from_db() -> list:
    cur = get_db().cursor()
    cur.execute("SELECT DISTINCT topic_text FROM topics ORDER BY topic_text LIMIT 500")
    return [row["topic_text"] for row in cur.fetchall()]


def load_topics_from_chapters(subject_id: str = None) -> list:
    """
    Load clean topic list from the chapters table.
    These are real NCERT chapter names — no noise.
    Optionally filter by subject.
    """
    cur = get_db().cursor()
    if subject_id:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0 AND subject_id = ?
            ORDER BY chapter_name
        """, (subject_id,))
    else:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0
            ORDER BY chapter_name
        """)
    return [row["chapter_name"] for row in cur.fetchall()]


# ─────────────────────────────────────────────
# DOWNLOAD & EXTRACT NCERT PDFs
# ─────────────────────────────────────────────
def extract_all_zips(folder: str) -> None:
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items()
                         if k.startswith("download_warning")), None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        st.error("❌ Downloaded file is not a valid ZIP.")
        st.stop()

    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)

    with st.spinner("📂 Extracting subject/chapter ZIPs…"):
        extract_all_zips(EXTRACT_DIR)

    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files

# ─────────────────────────────────────────────
# PDF UTILITIES
# ─────────────────────────────────────────────
def read_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except:
        return ""


def clean_text(text: str) -> str:
    text = re.sub(r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*", " ", text, flags=re.I)
    return re.sub(r"\s+", " ", text).strip()


@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts, paths = [], []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))
    return texts, paths

# ─────────────────────────────────────────────
# TOPICS — extracted from PDFs, cached in DB
# ─────────────────────────────────────────────

# ── Noise patterns to reject ──────────────────
# Bibliography entries: "SHARMA, R. 2005. Society..."
_AUTHOR_ENTRY   = re.compile(r"^[A-Z]{2,},\s+[A-Z]")
# Any line with a year like "1970." or "(2005)"
_HAS_YEAR       = re.compile(r"(19|20)\d{2}")
# Publisher / place names
_PUBLISHER      = re.compile(
    r"(press|publishing|publishers|edition|reprint|oxford|cambridge|mcgraw|"
    r"pearson|routledge|sage|wiley|elsevier|springer|penguin|harper|norton|"
    r"new york|new delhi|london|chicago|boston|mumbai|kolkata|chennai|"
    r"pp\.|vol\.|ibid|op\.cit|et al|isbn|doi|http|www\.)",
    re.I
)
# Lines that start with noise words
_NOISE_START    = re.compile(
    r"^(the|a|an|this|that|these|it|its|in|on|at|by|for|of|to|and|or|but|"
    r"such|also|thus|hence|therefore|however|moreover|furthermore|although|"
    r"figure|table|box|note|source|see|pg|pp|ref|ncert|reprint|let us|"
    r"activity|exercise|project|chapter|unit|section)",
    re.I
)
# Must look like a proper heading: starts uppercase, only letters/spaces/hyphens
_VALID_HEADING  = re.compile(r"^[A-Z][a-zA-Z\s\-:']{3,55}$")


def extract_topics_from_text(text: str) -> list:
    """
    Strict extractor — only picks real section/chapter headings from PDF text.

    Rules:
    - 2 to 6 words only
    - Must start with uppercase
    - Must be mostly title-cased (≥70% words capitalised)
    - No years, no author surname patterns, no publisher names
    - No punctuation endings, no digits
    - Rejects known noise start words
    """
    lines = text.split("\n")
    topics = set()

    for raw_line in lines:
        line = raw_line.strip()

        # ── Basic length filter ────────────────
        words = line.split()
        if not (2 <= len(words) <= 6):
            continue

        # ── Hard rejections ────────────────────
        if _AUTHOR_ENTRY.match(line):      # "SHARMA, R. 2005..."
            continue
        if _HAS_YEAR.search(line):         # anything with 1970, 2005 etc
            continue
        if _PUBLISHER.search(line):        # publisher / place names
            continue
        if _NOISE_START.match(line):       # starts with filler word
            continue
        if not _VALID_HEADING.match(line): # must match heading pattern
            continue

        # ── Endings filter ─────────────────────
        if line[-1] in ".,;:":
            continue
        last_word = words[-1].lower()
        if last_word in ("and", "or", "the", "of", "a", "an", "in", "on"):
            continue

        # ── Digits filter ─────────────────────
        if any(c.isdigit() for c in line):
            continue

        # ── Title-case check ──────────────────
        # At least 70% of words must start with uppercase
        titled = sum(1 for w in words if w and w[0].isupper())
        if titled / len(words) < 0.70:
            continue

        topics.add(line)

    return list(topics)


def topics_in_db() -> bool:
    """Quick check — are any topics already saved in the DB?"""
    cur = get_db().cursor()
    cur.execute("SELECT COUNT(*) as n FROM topics")
    return cur.fetchone()["n"] > 0


@st.cache_data(show_spinner=False)
def get_all_topics(subject_name: str = None) -> list:
    """
    Fast path: DB already has topics → pure SQL query, instant.
    Slow path: DB empty → extract from PDFs once, save, never slow again.
    """
    cur = get_db().cursor()

    # ── Fast path: topics already in DB ───────────────────────────────────
    if topics_in_db():
        if subject_name:
            subject_id = subject_name.lower().replace(" ", "_")
            cur.execute("""
                SELECT DISTINCT t.topic_text FROM topics t
                WHERE t.source_pdf IN (
                    SELECT DISTINCT pdf_filename FROM chapters
                    WHERE subject_id = ?
                )
                ORDER BY t.topic_text
            """, (subject_id,))
        else:
            cur.execute("""
                SELECT DISTINCT topic_text FROM topics
                ORDER BY topic_text LIMIT 600
            """)
        return [r["topic_text"] for r in cur.fetchall()]

    # ── Slow path: extract from PDFs (first time only) ────────────────────
    with st.spinner("🔍 Extracting topics from PDFs — first time only, won't repeat..."):
        texts, paths = load_all_texts()
        for text, path in zip(texts, paths):
            extracted = extract_topics_from_text(text)
            save_topics_to_db(extracted, path)

    # Now re-query with the fresh data
    return get_all_topics(subject_name)


# ─────────────────────────────────────────────
# EMBEDDER
# ─────────────────────────────────────────────
@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# ─────────────────────────────────────────────
# BOOT SEQUENCE
# ─────────────────────────────────────────────
_, pdf_files = download_and_extract()
# PDF embeddings and topics load lazily:
#   - topics: loaded when sidebar dropdown opens (DB query, instant if cached)
#   - embeddings: loaded when first search runs (cached after that)

# Session ID for anonymous history logging


# ─────────────────────────────────────────────
# SIDEBAR
# ─────────────────────────────────────────────
st.sidebar.header("🎓 Select Options")
# Loaded from DB — adding a new subject/level in DB instantly shows here
all_subjects     = get_subjects()
selected_subject = st.sidebar.selectbox("Select Subject", all_subjects)
all_levels       = get_levels(selected_subject)
selected_level   = st.sidebar.selectbox("Select Level", all_levels)

st.sidebar.markdown("---")

# ── Input mode toggle ──────────────────────────
input_mode = st.sidebar.radio(
    "Search by",
    ["📝 Type a query", "📋 Pick from topics"],
    help="Type your own question OR pick chapter names from the list"
)

user_query = ""
selected_topics = []

if input_mode == "📝 Type a query":
    user_query = st.sidebar.text_area(
        "Your question or keyword",
        placeholder="e.g. what is the caste system?\nor: federalism in India\nor: GDP and national income",
        height=100,
        help="Type anything — a question, keyword, or concept"
    )
    # derive topics list from query for YouTube/books (use query words)
    selected_topics = [w for w in user_query.split() if len(w) > 3] if user_query else []

else:
    subject_topics = get_all_topics(selected_subject)
    if not subject_topics:
        st.sidebar.warning("No topics found — PDFs will be scanned on first search.")
        selected_topics = []
    else:
        st.sidebar.caption(f"{len(subject_topics)} topics from {selected_subject} PDFs")
        selected_topics = st.sidebar.multiselect(
            "Select Topic(s)",
            subject_topics,
            help="Topics extracted from NCERT PDF text"
        )
        # Reset button — wipes DB cache and re-extracts from PDFs
        if st.sidebar.button("🔄 Reset & Re-extract Topics", help="Use if topics look noisy"):
            conn = get_db()
            conn.execute("DELETE FROM topics")
            conn.commit()
            st.cache_data.clear()
            st.rerun()
    # join selected topics into a query string for the embedder
    user_query = " ".join(selected_topics)

# Final search query used everywhere
search_query = user_query.strip()

# ─────────────────────────────────────────────
# CHAPTER RECOMMENDATIONS (embeddings)
# ─────────────────────────────────────────────
def lookup_chapter(pdf_filename: str, zip_source: str = None) -> dict:
    """
    Given a pdf_filename (and optionally zip_source for disambiguation),
    return readable chapter details from the DB.
    zip_source matters for Business Studies — Class 11 & 12 share identical filenames.
    Falls back to raw filename if not found.
    """
    cur = get_db().cursor()
    if zip_source:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name, book_name, is_special
            FROM chapters WHERE pdf_filename = ? AND zip_source = ?
        """, (pdf_filename, zip_source))
    else:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name, book_name, is_special
            FROM chapters WHERE pdf_filename = ? LIMIT 1
        """, (pdf_filename,))
    row = cur.fetchone()
    if row:
        return dict(row)
    return {
        "subject_id": "—", "class": "—", "chapter_number": "—",
        "chapter_name": pdf_filename, "book_name": "—", "is_special": 0
    }


@st.cache_resource(show_spinner="📐 Indexing PDF chapters (one-time)…")
def get_pdf_embeddings():
    """
    Encode ALL PDF texts ONCE and cache in memory.
    Any query then just encodes a single string and does cosine similarity — instant.
    """
    texts, paths = load_all_texts()
    embs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    return texts, paths, embs


def get_chapter_recommendations(query: str, top_n=5):
    """
    Search chapters using free text OR chapter name keywords.
    e.g. "what is the caste system", "GDP national income", "Federalism"
    PDF embeddings are pre-cached so each search is instant.
    """
    if not query or not query.strip():
        return pd.DataFrame()

    texts, paths, text_emb = get_pdf_embeddings()   # cached — no recompute
    topic_emb = embedder.encode([query.strip()], convert_to_numpy=True)
    sims      = cosine_similarity(topic_emb, text_emb).flatten()
    top_idx   = sims.argsort()[-top_n:][::-1]

    rows = []
    for i in top_idx:
        info = lookup_chapter(paths[i])
        if info.get("is_special"):
            continue
        rows.append({
            "Subject":      info["subject_id"].replace("_", " ").title(),
            "Class":        f"Class {info['class']}",
            "Book":         info["book_name"],
            "Ch No.":       info["chapter_number"],
            "Chapter Name": info["chapter_name"],
            "Relevance":    round(float(sims[i]), 4),
        })
    return pd.DataFrame(rows) if rows else pd.DataFrame()

# ─────────────────────────────────────────────
# BOOK & PAPER RECOMMENDATIONS (from DB)
# ─────────────────────────────────────────────
def recommend_materials(subject: str, level: str, topics: tuple) -> pd.DataFrame:
    if not topics:
        return pd.DataFrame({"Message": ["Please select at least one topic"]})
    rows = get_recommendations(subject, level)
    if not rows:
        return pd.DataFrame({"Message": [f"No data found for {subject} / {level}"]})
    return pd.DataFrame([{
        "Type":          r["level_type"],
        "Title":         r["title"],
        "Author":        r["author"],
        "Why Read This": r["why"],
        "NCERT Chapter": r["ncert_chapter_link"] or "—",
        "Link":          r["link"],
    } for r in rows])

# ─────────────────────────────────────────────
# YOUTUBE SEARCH URL BUILDER
# ─────────────────────────────────────────────
def build_youtube_url(subject: str, level: str, topics: list) -> str:
    """
    Build a YouTube search URL from subject + level + selected topics.
    Opens YouTube search results directly — no API key needed.
    """
    import urllib.parse
    # Compose a focused search query
    topic_str = " ".join(topics[:3]) if topics else ""
    query = f"NCERT {subject} {level} {topic_str} class 11 12".strip()
    encoded = urllib.parse.quote_plus(query)
    return f"https://www.youtube.com/results?search_query={encoded}"


# ─────────────────────────────────────────────
# RENDER — Tabs layout
# ─────────────────────────────────────────────
st.markdown("---")

tab1, tab2, tab3 = st.tabs([
    "📄 Chapter Recommendations",
    "📚 Books & Research Papers",
    "▶️ YouTube Videos",
])

# ── Tab 1: Chapter Recommendations ────────────
with tab1:
    st.markdown(
        "Finds the most relevant NCERT chapters based on your query. "
        "Works with free text — *what is caste system*, *explain GDP*, anything."
    )
    if not search_query:
        st.info("👈 Type a question or select chapters from the sidebar to search.")
    else:
        st.markdown(f"🔎 **Searching for:** `{search_query}`")
        # Auto-search on every new query — no button needed
        with st.spinner("Finding relevant chapters..."):
            df = get_chapter_recommendations(search_query)
        if df.empty:
            st.warning("No matching chapters found. Try different keywords.")
        else:
            st.success(f"✅ Top {len(df)} chapters found!")
            st.dataframe(
                df,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Relevance": st.column_config.ProgressColumn(
                        "Relevance",
                        min_value=0,
                        max_value=1,
                        format="%.4f"
                    )
                }
            )

# ── Tab 2: Books & Research Papers ────────────
with tab2:
    st.markdown(
        f"Showing **{selected_level}** level resources for **{selected_subject}** "
        f"— NCERT textbooks → reference books → research papers."
    )
    if st.button("📖 Get Books & Papers", type="primary", key="book_btn"):

        rows = get_recommendations(selected_subject, selected_level)
        if not rows:
            st.warning(f"No recommendations found for {selected_subject} / {selected_level}")
        else:
            for r in rows:
                type_icons = {"NCERT": "📗", "Book": "📘", "Paper": "📄"}
                icon = type_icons.get(r["level_type"], "📌")
                with st.expander(f"{icon} {r['title']} — *{r['author']}*", expanded=False):
                    st.markdown(f"**Type:** {r['level_type']}")
                    if r["journal"]:
                        st.markdown(f"**Journal:** {r['journal']}")
                    if r["ncert_chapter_link"]:
                        st.markdown(f"**NCERT Chapter:** `{r['ncert_chapter_link']}`")
                    st.markdown(f"💡 *{r['why']}*")
                    if r["link"]:
                        st.markdown(f"[🔗 Open Resource]({r['link']})")

# ── Tab 3: YouTube Videos ──────────────────────
with tab3:
    st.markdown(
        "Clicking the button below will open a **YouTube search** in a new tab "
        "with a query built from your selected subject, level, and topics."
    )

    # Show the query that will be used
    yt_query_preview = f"NCERT {selected_subject} {selected_level} {search_query} class 11 12".strip()
    st.markdown(f"**Search query:** `{yt_query_preview}`")

    if not search_query:
        st.info("👈 Type a query or select topics to refine the YouTube search.")

    # Always show the button — even without topics it gives subject+level results
    yt_url = build_youtube_url(selected_subject, selected_level, [search_query])
    st.link_button("▶️ Search on YouTube", yt_url, type="primary")

    st.markdown("---")
    # Also show quick-access searches for each level
    st.markdown("**Or jump directly to a level:**")
    level_icons = {"Beginner": "🌱", "Intermediate": "📚", "Advanced": "🔬"}
    yt_levels = get_levels(selected_subject)
    cols = st.columns(len(yt_levels))
    for col, lvl in zip(cols, yt_levels):
        with col:
            icon = level_icons.get(lvl, "▶️")
            url  = build_youtube_url(selected_subject, lvl, [search_query])
            st.link_button(f"{icon} {lvl}", url, use_container_width=True)



Overwriting app.py


ADDED EMBEDDINGS TO THE DB

In [3]:
# @title
%%writefile app.py
import os
import re
import uuid
import hashlib
import zipfile
from pathlib import Path

import numpy as np
import streamlit as st
import gdown
import requests
import pandas as pd
import sqlite3
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
FILE_ID    = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH   = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"
DB_PATH    = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"

# ─────────────────────────────────────────────
# PAGE CONFIG
# ─────────────────────────────────────────────
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and research paper recommendations for Class 11–12 students")

# ─────────────────────────────────────────────
# MOUNT GOOGLE DRIVE
# ─────────────────────────────────────────────
try:
    from google.colab import drive
    if not os.path.exists("/content/drive/MyDrive"):
        drive.mount("/content/drive")
except ImportError:
    DB_PATH = "ncert.db"   # fallback for local runs

# ─────────────────────────────────────────────
# DATABASE — connect only, no table creation
# ─────────────────────────────────────────────
@st.cache_resource
def get_db():
    if not os.path.exists(DB_PATH):
        st.error(
            f"❌ Database not found at:\n`{DB_PATH}`\n\n"
            "Please run setup_db.ipynb first, or update DB_PATH at the top of app.py."
        )
        st.stop()
    try:
        conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        conn.row_factory = sqlite3.Row
        # Sanity check — if file is corrupt/not a DB this will raise immediately
        conn.execute("SELECT 1")
        # Ensure the embeddings table exists (safe to run every time — no-op if already there)
        conn.execute("""
            CREATE TABLE IF NOT EXISTS pdf_embeddings (
                pdf_filename  TEXT PRIMARY KEY,
                embedding     BLOB NOT NULL,
                text_hash     TEXT NOT NULL,
                created_at    TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        conn.commit()
        return conn
    except sqlite3.DatabaseError as e:
        st.error(
            f"❌ File exists but is not a valid SQLite database:\n`{DB_PATH}`\n\n"
            f"Error: {e}\n\nPlease re-run setup_db.ipynb to recreate it."
        )
        st.stop()


# ── Subject / Level helpers ──────────────────

def get_subjects() -> list:
    cur = get_db().cursor()
    cur.execute("SELECT subject_name FROM subjects ORDER BY subject_name")
    return [row["subject_name"] for row in cur.fetchall()]


def get_levels(subject_name: str = None) -> list:
    cur = get_db().cursor()
    level_order = (
        "CASE level WHEN 'Beginner' THEN 1 "
        "WHEN 'Intermediate' THEN 2 WHEN 'Advanced' THEN 3 ELSE 4 END"
    )
    if subject_name:
        subject_id = subject_name.lower().replace(" ", "_")
        cur.execute(
            f"SELECT DISTINCT level FROM recommendations "
            f"WHERE subject_id = ? ORDER BY {level_order}",
            (subject_id,)
        )
    else:
        cur.execute(f"SELECT DISTINCT level FROM recommendations ORDER BY {level_order}")
    return [row["level"] for row in cur.fetchall()]


def get_recommendations(subject_name: str, level: str) -> list:
    subject_id = subject_name.lower().replace(" ", "_")
    cur = get_db().cursor()
    cur.execute("""
        SELECT title, author, level_type, why, link, journal, ncert_chapter_link
        FROM recommendations
        WHERE subject_id = ? AND level = ?
        ORDER BY rec_id
    """, (subject_id, level))
    return [dict(row) for row in cur.fetchall()]


def save_topics_to_db(topics: list, source_pdf: str) -> None:
    conn = get_db()
    conn.executemany(
        "INSERT OR IGNORE INTO topics (topic_text, source_pdf) VALUES (?, ?)",
        [(t, source_pdf) for t in topics]
    )
    conn.commit()


def load_topics_from_db() -> list:
    cur = get_db().cursor()
    cur.execute("SELECT DISTINCT topic_text FROM topics ORDER BY topic_text LIMIT 500")
    return [row["topic_text"] for row in cur.fetchall()]


def load_topics_from_chapters(subject_id: str = None) -> list:
    cur = get_db().cursor()
    if subject_id:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0 AND subject_id = ?
            ORDER BY chapter_name
        """, (subject_id,))
    else:
        cur.execute("""
            SELECT DISTINCT chapter_name FROM chapters
            WHERE is_special = 0
            ORDER BY chapter_name
        """)
    return [row["chapter_name"] for row in cur.fetchall()]


# ─────────────────────────────────────────────
# DOWNLOAD & EXTRACT NCERT PDFs
# ─────────────────────────────────────────────

def extract_all_zips(folder: str) -> None:
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    found_new = True
                except zipfile.BadZipFile:
                    st.warning(f"⚠️ Skipping corrupt ZIP: {file}")
                except Exception as e:
                    st.warning(f"⚠️ Could not extract {file}: {e}")


@st.cache_data
def download_and_extract():
    if not os.path.exists(ZIP_PATH):
        with st.spinner("⬇️ Downloading NCERT dataset…"):
            downloaded = False
            try:
                gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
                downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
            except Exception as e:
                st.warning(f"gdown failed: {e}. Trying fallback…")

            if not downloaded:
                try:
                    session  = requests.Session()
                    URL      = "https://drive.google.com/uc?export=download"
                    response = session.get(URL, params={"id": FILE_ID}, stream=True)
                    token    = next(
                        (v for k, v in response.cookies.items() if k.startswith("download_warning")),
                        None
                    )
                    if token:
                        response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
                    else:
                        response = session.get(
                            f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                            stream=True,
                        )
                    with open(ZIP_PATH, "wb") as f:
                        for chunk in response.iter_content(chunk_size=32_768):
                            if chunk:
                                f.write(chunk)
                    if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                        raise ValueError("Downloaded file too small.")
                except Exception as e2:
                    st.error(f"❌ Both download methods failed: {e2}")
                    st.stop()

        if not zipfile.is_zipfile(ZIP_PATH):
            os.remove(ZIP_PATH)
            st.error("❌ Downloaded file is not a valid ZIP.")
            st.stop()

    if not os.path.exists(EXTRACT_DIR):
        with st.spinner("📦 Extracting outer ZIP…"):
            with zipfile.ZipFile(ZIP_PATH, "r") as zf:
                zf.extractall(EXTRACT_DIR)
        with st.spinner("📂 Extracting subject/chapter ZIPs…"):
            extract_all_zips(EXTRACT_DIR)

    pdf_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(EXTRACT_DIR)
        for f in files
        if f.lower().endswith(".pdf")
    ]
    st.info(f"📄 PDFs found: {len(pdf_files)}")
    return EXTRACT_DIR, pdf_files


# ─────────────────────────────────────────────
# PDF UTILITIES
# ─────────────────────────────────────────────

def read_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except Exception:
        return ""


def clean_text(text: str) -> str:
    text = re.sub(
        r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*",
        " ", text, flags=re.I
    )
    return re.sub(r"\s+", " ", text).strip()


@st.cache_data(show_spinner="📖 Reading PDFs…")
def load_all_texts():
    texts, paths = [], []
    for pdf in Path(EXTRACT_DIR).rglob("*.pdf"):
        t = clean_text(read_pdf(str(pdf)))
        if len(t.split()) > 50:
            texts.append(t)
            paths.append(str(pdf.name))
    return texts, paths


# ─────────────────────────────────────────────
# TOPICS — extracted from PDFs, cached in DB
# ─────────────────────────────────────────────

_AUTHOR_ENTRY  = re.compile(r"^[A-Z]{2,},\s+[A-Z]")
_HAS_YEAR      = re.compile(r"(19|20)\d{2}")
_PUBLISHER     = re.compile(
    r"(press|publishing|publishers|edition|reprint|oxford|cambridge|mcgraw|"
    r"pearson|routledge|sage|wiley|elsevier|springer|penguin|harper|norton|"
    r"new york|new delhi|london|chicago|boston|mumbai|kolkata|chennai|"
    r"pp\.|vol\.|ibid|op\.cit|et al|isbn|doi|http|www\.)", re.I
)
_NOISE_START   = re.compile(
    r"^(the|a|an|this|that|these|it|its|in|on|at|by|for|of|to|and|or|but|"
    r"such|also|thus|hence|therefore|however|moreover|furthermore|although|"
    r"figure|table|box|note|source|see|pg|pp|ref|ncert|reprint|let us|"
    r"activity|exercise|project|chapter|unit|section)", re.I
)
_VALID_HEADING = re.compile(r"^[A-Z][a-zA-Z\s\-:']{3,55}$")


def extract_topics_from_text(text: str) -> list:
    lines  = text.split("\n")
    topics = set()
    for raw_line in lines:
        line  = raw_line.strip()
        words = line.split()
        if not (2 <= len(words) <= 6):
            continue
        if _AUTHOR_ENTRY.match(line):
            continue
        if _HAS_YEAR.search(line):
            continue
        if _PUBLISHER.search(line):
            continue
        if _NOISE_START.match(line):
            continue
        if not _VALID_HEADING.match(line):
            continue
        if line[-1] in ".,;:":
            continue
        last_word = words[-1].lower()
        if last_word in ("and", "or", "the", "of", "a", "an", "in", "on"):
            continue
        if any(c.isdigit() for c in line):
            continue
        titled = sum(1 for w in words if w and w[0].isupper())
        if titled / len(words) < 0.70:
            continue
        topics.add(line)
    return list(topics)


def topics_in_db() -> bool:
    cur = get_db().cursor()
    cur.execute("SELECT COUNT(*) as n FROM topics")
    return cur.fetchone()["n"] > 0


@st.cache_data(show_spinner=False)
def get_all_topics(subject_name: str = None) -> list:
    cur = get_db().cursor()

    if topics_in_db():
        if subject_name:
            subject_id = subject_name.lower().replace(" ", "_")
            cur.execute("""
                SELECT DISTINCT t.topic_text FROM topics t
                WHERE t.source_pdf IN (
                    SELECT DISTINCT pdf_filename FROM chapters WHERE subject_id = ?
                )
                ORDER BY t.topic_text
            """, (subject_id,))
        else:
            cur.execute("""
                SELECT DISTINCT topic_text FROM topics
                ORDER BY topic_text LIMIT 600
            """)
        return [r["topic_text"] for r in cur.fetchall()]

    with st.spinner("🔍 Extracting topics from PDFs — first time only, won't repeat..."):
        texts, paths = load_all_texts()
        for text, path in zip(texts, paths):
            extracted = extract_topics_from_text(text)
            save_topics_to_db(extracted, path)

    return get_all_topics(subject_name)


# ─────────────────────────────────────────────
# EMBEDDER
# ─────────────────────────────────────────────

@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()


# ─────────────────────────────────────────────
# EMBEDDING HELPERS — DB backed
# ─────────────────────────────────────────────

def _text_hash(text: str) -> str:
    """MD5 of PDF text — used to detect stale cached embeddings."""
    return hashlib.md5(text.encode()).hexdigest()


def _load_embedding_from_db(pdf_filename: str, text: str):
    """
    Returns the cached numpy embedding if it exists AND the text hasn't changed.
    Returns None if not cached or stale.
    """
    try:
        cur = get_db().cursor()
        cur.execute(
            "SELECT embedding, text_hash FROM pdf_embeddings WHERE pdf_filename = ?",
            (pdf_filename,)
        )
        row = cur.fetchone()
        if row and row["text_hash"] == _text_hash(text):
            return np.frombuffer(row["embedding"], dtype=np.float32)
    except Exception:
        pass
    return None


def _save_embedding_to_db(pdf_filename: str, text: str, embedding: np.ndarray) -> None:
    """Upsert a single embedding into the DB."""
    try:
        get_db().execute("""
            INSERT OR REPLACE INTO pdf_embeddings (pdf_filename, embedding, text_hash)
            VALUES (?, ?, ?)
        """, (pdf_filename, embedding.astype(np.float32).tobytes(), _text_hash(text)))
        get_db().commit()
    except Exception as e:
        st.warning(f"⚠️ Could not save embedding for {pdf_filename}: {e}")


# ─────────────────────────────────────────────
# BOOT SEQUENCE
# ─────────────────────────────────────────────

_, pdf_files = download_and_extract()


# ─────────────────────────────────────────────
# CHAPTER RECOMMENDATIONS (embeddings)
# ─────────────────────────────────────────────

def lookup_chapter(pdf_filename: str, zip_source: str = None) -> dict:
    cur = get_db().cursor()
    if zip_source:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name,
                   book_name, is_special
            FROM chapters
            WHERE pdf_filename = ? AND zip_source = ?
        """, (pdf_filename, zip_source))
    else:
        cur.execute("""
            SELECT subject_id, class, chapter_number, chapter_name,
                   book_name, is_special
            FROM chapters WHERE pdf_filename = ? LIMIT 1
        """, (pdf_filename,))
    row = cur.fetchone()
    if row:
        return dict(row)
    return {
        "subject_id": "—", "class": "—", "chapter_number": "—",
        "chapter_name": pdf_filename, "book_name": "—", "is_special": 0
    }


@st.cache_resource(show_spinner="📐 Indexing PDF chapters (one-time)…")
def get_pdf_embeddings():
    """
    Smart embedding loader:
      • Checks DB first for each PDF.
      • Only encodes PDFs that are NEW or whose text has CHANGED.
      • Saves new embeddings to DB so every subsequent run is instant.

    After the first full run, this function is O(n) DB reads — no GPU/CPU encoding.
    """
    texts, paths = load_all_texts()

    embeddings       = [None] * len(texts)
    to_encode_idx    = []
    to_encode_texts  = []

    # ── Pass 1: load cached embeddings from DB ──────────────────────────────
    for i, (text, path) in enumerate(zip(texts, paths)):
        cached = _load_embedding_from_db(path, text)
        if cached is not None:
            embeddings[i] = cached          # ✅ cache hit — no encoding needed
        else:
            to_encode_idx.append(i)         # ❌ missing or stale — queue for encoding
            to_encode_texts.append(text)

    # ── Pass 2: encode only missing/stale PDFs ──────────────────────────────
    if to_encode_texts:
        status_msg = (
            f"🧠 Encoding {len(to_encode_texts)} "
            f"{'new/changed' if len(to_encode_texts) < len(texts) else ''} PDFs…"
        )
        with st.spinner(status_msg):
            new_embs = embedder.encode(
                to_encode_texts,
                convert_to_numpy=True,
                show_progress_bar=False
            )
        # Save each new embedding to DB and fill the array
        for idx, emb, text in zip(to_encode_idx, new_embs, to_encode_texts):
            embeddings[idx] = emb.astype(np.float32)
            _save_embedding_to_db(paths[idx], text, emb)

    return texts, paths, np.array(embeddings, dtype=np.float32)


def get_chapter_recommendations(query: str, top_n: int = 5) -> pd.DataFrame:
    if not query or not query.strip():
        return pd.DataFrame()

    texts, paths, text_emb = get_pdf_embeddings()   # instant after first run
    topic_emb = embedder.encode([query.strip()], convert_to_numpy=True)
    sims      = cosine_similarity(topic_emb, text_emb).flatten()
    top_idx   = sims.argsort()[-top_n:][::-1]

    rows = []
    for i in top_idx:
        info = lookup_chapter(paths[i])
        if info.get("is_special"):
            continue
        rows.append({
            "Subject":      info["subject_id"].replace("_", " ").title(),
            "Class":        f"Class {info['class']}",
            "Book":         info["book_name"],
            "Ch No.":       info["chapter_number"],
            "Chapter Name": info["chapter_name"],
            "Relevance":    round(float(sims[i]), 4),
        })
    return pd.DataFrame(rows) if rows else pd.DataFrame()


# ─────────────────────────────────────────────
# BOOK & PAPER RECOMMENDATIONS (from DB)
# ─────────────────────────────────────────────

def recommend_materials(subject: str, level: str, topics: tuple) -> pd.DataFrame:
    if not topics:
        return pd.DataFrame({"Message": ["Please select at least one topic"]})
    rows = get_recommendations(subject, level)
    if not rows:
        return pd.DataFrame({"Message": [f"No data found for {subject} / {level}"]})
    return pd.DataFrame([{
        "Type":          r["level_type"],
        "Title":         r["title"],
        "Author":        r["author"],
        "Why Read This": r["why"],
        "NCERT Chapter": r["ncert_chapter_link"] or "—",
        "Link":          r["link"],
    } for r in rows])


# ─────────────────────────────────────────────
# YOUTUBE SEARCH URL BUILDER
# ─────────────────────────────────────────────

def build_youtube_url(subject: str, level: str, topics: list) -> str:
    import urllib.parse
    topic_str = " ".join(topics[:3]) if topics else ""
    query     = f"NCERT {subject} {level} {topic_str} class 11 12".strip()
    encoded   = urllib.parse.quote_plus(query)
    return f"https://www.youtube.com/results?search_query={encoded}"


# ─────────────────────────────────────────────
# SIDEBAR
# ─────────────────────────────────────────────

st.sidebar.header("🎓 Select Options")

all_subjects     = get_subjects()
selected_subject = st.sidebar.selectbox("Select Subject", all_subjects)

all_levels       = get_levels(selected_subject)
selected_level   = st.sidebar.selectbox("Select Level", all_levels)

st.sidebar.markdown("---")

input_mode = st.sidebar.radio(
    "Search by",
    ["📝 Type a query", "📋 Pick from topics"],
    help="Type your own question OR pick chapter names from the list"
)

user_query      = ""
selected_topics = []

if input_mode == "📝 Type a query":
    user_query = st.sidebar.text_area(
        "Your question or keyword",
        placeholder=(
            "e.g. what is the caste system?\n"
            "or: federalism in India\n"
            "or: GDP and national income"
        ),
        height=100,
        help="Type anything — a question, keyword, or concept"
    )
    selected_topics = [w for w in user_query.split() if len(w) > 3] if user_query else []

else:
    subject_topics = get_all_topics(selected_subject)
    if not subject_topics:
        st.sidebar.warning("No topics found — PDFs will be scanned on first search.")
        selected_topics = []
    else:
        st.sidebar.caption(f"{len(subject_topics)} topics from {selected_subject} PDFs")
        selected_topics = st.sidebar.multiselect(
            "Select Topic(s)", subject_topics,
            help="Topics extracted from NCERT PDF text"
        )

user_query   = " ".join(selected_topics)
search_query = user_query.strip()


# ─────────────────────────────────────────────
# RENDER — Tabs layout
# ─────────────────────────────────────────────

st.markdown("---")
tab1, tab2, tab3 = st.tabs([
    "📄 Chapter Recommendations",
    "📚 Books & Research Papers",
    "▶️ YouTube Videos",
])

# ── Tab 1: Chapter Recommendations ────────────
with tab1:
    st.markdown(
        "Finds the most relevant NCERT chapters based on your query. "
        "Works with free text — *what is caste system*, *explain GDP*, anything."
    )
    if not search_query:
        st.info("👈 Type a question or select chapters from the sidebar to search.")
    else:
        st.markdown(f"🔎 **Searching for:** `{search_query}`")
        with st.spinner("Finding relevant chapters..."):
            df = get_chapter_recommendations(search_query)
        if df.empty:
            st.warning("No matching chapters found. Try different keywords.")
        else:
            st.success(f"✅ Top {len(df)} chapters found!")
            st.dataframe(
                df,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Relevance": st.column_config.ProgressColumn(
                        "Relevance", min_value=0, max_value=1, format="%.4f"
                    )
                }
            )

# ── Tab 2: Books & Research Papers ────────────
with tab2:
    st.markdown(
        f"Showing **{selected_level}** level resources for **{selected_subject}** "
        f"— NCERT textbooks → reference books → research papers."
    )
    if st.button("📖 Get Books & Papers", type="primary", key="book_btn"):
        rows = get_recommendations(selected_subject, selected_level)
        if not rows:
            st.warning(f"No recommendations found for {selected_subject} / {selected_level}")
        else:
            for r in rows:
                type_icons = {"NCERT": "📗", "Book": "📘", "Paper": "📄"}
                icon = type_icons.get(r["level_type"], "📌")
                with st.expander(f"{icon} {r['title']} — *{r['author']}*", expanded=False):
                    st.markdown(f"**Type:** {r['level_type']}")
                    if r["journal"]:
                        st.markdown(f"**Journal:** {r['journal']}")
                    if r["ncert_chapter_link"]:
                        st.markdown(f"**NCERT Chapter:** `{r['ncert_chapter_link']}`")
                    st.markdown(f"💡 *{r['why']}*")
                    if r["link"]:
                        st.markdown(f"[🔗 Open Resource]({r['link']})")

# ── Tab 3: YouTube Videos ─────────────────────
with tab3:
    st.markdown(
        "Clicking the button below will open a **YouTube search** in a new tab "
        "with a query built from your selected subject, level, and topics."
    )
    yt_query_preview = f"NCERT {selected_subject} {selected_level} {search_query} class 11 12".strip()
    st.markdown(f"**Search query:** `{yt_query_preview}`")

    if not search_query:
        st.info("👈 Type a query or select topics to refine the YouTube search.")

    yt_url = build_youtube_url(selected_subject, selected_level, [search_query])
    st.link_button("▶️ Search on YouTube", yt_url, type="primary")

    st.markdown("---")
    st.markdown("**Or jump directly to a level:**")

    level_icons = {"Beginner": "🌱", "Intermediate": "📚", "Advanced": "🔬"}
    yt_levels   = get_levels(selected_subject)
    cols        = st.columns(len(yt_levels))
    for col, lvl in zip(cols, yt_levels):
        with col:
            icon = level_icons.get(lvl, "▶️")
            url  = build_youtube_url(selected_subject, lvl, [search_query])
            st.link_button(f"{icon} {lvl}", url, use_container_width=True)

Writing app.py


removed the reading pdfs, everything happens by fetching frilies from DB...all embeddings stored in DB

In [7]:
%%writefile app.py
import os
import re
import hashlib
import urllib.parse

import numpy as np
import streamlit as st
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
DB_PATH = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"

# ─────────────────────────────────────────────
# PAGE CONFIG
# ─────────────────────────────────────────────
st.set_page_config(page_title="NCERT Learning Assistant", layout="wide")
st.title("📘 NCERT Learning Assistant")
st.write("Select topics to get chapter, book, and research paper recommendations for Class 11–12 students")

# ─────────────────────────────────────────────
# MOUNT GOOGLE DRIVE
# ─────────────────────────────────────────────
try:
    from google.colab import drive
    if not os.path.exists("/content/drive/MyDrive"):
        drive.mount("/content/drive")
except ImportError:
    DB_PATH = "ncert.db"  # fallback for local runs

# ─────────────────────────────────────────────
# DATABASE
# ─────────────────────────────────────────────
@st.cache_resource
def get_db():
    if not os.path.exists(DB_PATH):
        st.error(
            f"❌ Database not found at:\n`{DB_PATH}`\n\n"
            "Please run setup_db.ipynb first, or update DB_PATH at the top of app.py."
        )
        st.stop()
    try:
        conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        conn.row_factory = sqlite3.Row
        conn.execute("SELECT 1")  # sanity check
        return conn
    except sqlite3.DatabaseError as e:
        st.error(
            f"❌ File exists but is not a valid SQLite database:\n`{DB_PATH}`\n\n"
            f"Error: {e}\n\nPlease re-run setup_db.ipynb to recreate it."
        )
        st.stop()

# ─────────────────────────────────────────────
# DB HELPERS
# ─────────────────────────────────────────────
def get_subjects() -> list:
    cur = get_db().cursor()
    cur.execute("SELECT subject_name FROM subjects ORDER BY subject_name")
    return [row["subject_name"] for row in cur.fetchall()]


def get_levels(subject_name: str = None) -> list:
    cur = get_db().cursor()
    level_order = (
        "CASE level WHEN 'Beginner' THEN 1 "
        "WHEN 'Intermediate' THEN 2 WHEN 'Advanced' THEN 3 ELSE 4 END"
    )
    if subject_name:
        subject_id = subject_name.lower().replace(" ", "_")
        cur.execute(
            f"SELECT DISTINCT level FROM recommendations "
            f"WHERE subject_id = ? ORDER BY {level_order}",
            (subject_id,)
        )
    else:
        cur.execute(f"SELECT DISTINCT level FROM recommendations ORDER BY {level_order}")
    return [row["level"] for row in cur.fetchall()]


def get_recommendations(subject_name: str, level: str) -> list:
    subject_id = subject_name.lower().replace(" ", "_")
    cur = get_db().cursor()
    cur.execute("""
        SELECT title, author, level_type, why, link, journal, ncert_chapter_link
        FROM recommendations
        WHERE subject_id = ? AND level = ?
        ORDER BY rec_id
    """, (subject_id, level))
    return [dict(row) for row in cur.fetchall()]


def get_all_topics(subject_name: str = None) -> list:
    cur = get_db().cursor()
    if subject_name:
        subject_id = subject_name.lower().replace(" ", "_")
        cur.execute("""
            SELECT DISTINCT t.topic_text FROM topics t
            WHERE t.source_pdf IN (
                SELECT DISTINCT pdf_filename FROM chapters WHERE subject_id = ?
            )
            ORDER BY t.topic_text
        """, (subject_id,))
    else:
        cur.execute("SELECT DISTINCT topic_text FROM topics ORDER BY topic_text LIMIT 600")
    return [r["topic_text"] for r in cur.fetchall()]


def lookup_chapter(pdf_filename: str) -> dict:
    cur = get_db().cursor()
    cur.execute("""
        SELECT subject_id, class, chapter_number, chapter_name, book_name, is_special
        FROM chapters WHERE pdf_filename = ? LIMIT 1
    """, (pdf_filename,))
    row = cur.fetchone()
    if row:
        return dict(row)
    return {
        "subject_id": "—", "class": "—", "chapter_number": "—",
        "chapter_name": pdf_filename, "book_name": "—", "is_special": 0
    }

# ─────────────────────────────────────────────
# EMBEDDER
# ─────────────────────────────────────────────
@st.cache_resource(show_spinner="🤖 Loading embedding model…")
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

embedder = load_embedder()

# ─────────────────────────────────────────────
# LOAD ALL EMBEDDINGS FROM DB (once per session)
# ─────────────────────────────────────────────
@st.cache_resource(show_spinner="⚡ Loading embeddings from database…")
def load_embeddings_from_db():
    """
    Reads ALL embeddings + filenames straight from DB into numpy arrays.
    No PDF reading. No encoding. Pure DB → RAM.
    Runs once per session in ~1-2 seconds.
    """
    cur = get_db().cursor()
    cur.execute("SELECT pdf_filename, embedding FROM pdf_embeddings")
    rows = cur.fetchall()

    paths      = []
    embeddings = []
    for row in rows:
        emb = np.frombuffer(row["embedding"], dtype=np.float32)
        paths.append(row["pdf_filename"])
        embeddings.append(emb)

    return paths, np.array(embeddings, dtype=np.float32)

# ─────────────────────────────────────────────
# CHAPTER RECOMMENDATIONS
# ─────────────────────────────────────────────
def get_chapter_recommendations(query: str, top_n: int = 5) -> pd.DataFrame:
    if not query or not query.strip():
        return pd.DataFrame()

    paths, text_emb = load_embeddings_from_db()   # instant — already in RAM
    query_emb = embedder.encode([query.strip()], convert_to_numpy=True)
    sims      = cosine_similarity(query_emb, text_emb).flatten()
    top_idx   = sims.argsort()[-top_n:][::-1]

    rows = []
    for i in top_idx:
        info = lookup_chapter(paths[i])
        if info.get("is_special"):
            continue
        rows.append({
            "Subject":      info["subject_id"].replace("_", " ").title(),
            "Class":        f"Class {info['class']}",
            "Book":         info["book_name"],
            "Ch No.":       info["chapter_number"],
            "Chapter Name": info["chapter_name"],
            "Relevance":    round(float(sims[i]), 4),
        })
    return pd.DataFrame(rows) if rows else pd.DataFrame()

# ─────────────────────────────────────────────
# YOUTUBE URL BUILDER
# ─────────────────────────────────────────────
def build_youtube_url(subject: str, level: str, topics: list) -> str:
    topic_str = " ".join(topics[:3]) if topics else ""
    query     = f"NCERT {subject} {level} {topic_str} class 11 12".strip()
    return f"https://www.youtube.com/results?search_query={urllib.parse.quote_plus(query)}"

# ─────────────────────────────────────────────
# SIDEBAR
# ─────────────────────────────────────────────
st.sidebar.header("🎓 Select Options")

all_subjects     = get_subjects()
selected_subject = st.sidebar.selectbox("Select Subject", all_subjects)

all_levels       = get_levels(selected_subject)
selected_level   = st.sidebar.selectbox("Select Level", all_levels)

st.sidebar.markdown("---")

input_mode = st.sidebar.radio(
    "Search by",
    ["📝 Type a query", "📋 Pick from topics"],
    help="Type your own question OR pick chapter names from the list"
)

user_query      = ""
selected_topics = []

if input_mode == "📝 Type a query":
    user_query = st.sidebar.text_area(
        "Your question or keyword",
        placeholder=(
            "e.g. what is the caste system?\n"
            "or: federalism in India\n"
            "or: GDP and national income"
        ),
        height=100,
        help="Type anything — a question, keyword, or concept"
    )
    selected_topics = [w for w in user_query.split() if len(w) > 3] if user_query else []

else:
    subject_topics = get_all_topics(selected_subject)
    if not subject_topics:
        st.sidebar.warning("No topics found in database.")
        selected_topics = []
    else:
        st.sidebar.caption(f"{len(subject_topics)} topics available")
        selected_topics = st.sidebar.multiselect(
            "Select Topic(s)", subject_topics,
            help="Topics from NCERT chapters"
        )
    user_query = " ".join(selected_topics)

search_query = user_query.strip()

# ─────────────────────────────────────────────
# TABS
# ─────────────────────────────────────────────
st.markdown("---")
tab1, tab2, tab3 = st.tabs([
    "📄 Chapter Recommendations",
    "📚 Books & Research Papers",
    "▶️ YouTube Videos",
])

# ── Tab 1: Chapter Recommendations ────────────
with tab1:
    st.markdown(
        "Finds the most relevant NCERT chapters based on your query. "
        "Works with free text — *what is caste system*, *explain GDP*, anything."
    )
    if not search_query:
        st.info("👈 Type a question or select topics from the sidebar to search.")
    else:
        st.markdown(f"🔎 **Searching for:** `{search_query}`")
        with st.spinner("Finding relevant chapters..."):
            df = get_chapter_recommendations(search_query)
        if df.empty:
            st.warning("No matching chapters found. Try different keywords.")
        else:
            st.success(f"✅ Top {len(df)} chapters found!")
            st.dataframe(
                df,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Relevance": st.column_config.ProgressColumn(
                        "Relevance", min_value=0, max_value=1, format="%.4f"
                    )
                }
            )

# ── Tab 2: Books & Research Papers ────────────
with tab2:
    st.markdown(
        f"Showing **{selected_level}** level resources for **{selected_subject}** "
        f"— NCERT textbooks → reference books → research papers."
    )
    if st.button("📖 Get Books & Papers", type="primary", key="book_btn"):
        rows = get_recommendations(selected_subject, selected_level)
        if not rows:
            st.warning(f"No recommendations found for {selected_subject} / {selected_level}")
        else:
            for r in rows:
                type_icons = {"NCERT": "📗", "Book": "📘", "Paper": "📄"}
                icon = type_icons.get(r["level_type"], "📌")
                with st.expander(f"{icon} {r['title']} — *{r['author']}*", expanded=False):
                    st.markdown(f"**Type:** {r['level_type']}")
                    if r["journal"]:
                        st.markdown(f"**Journal:** {r['journal']}")
                    if r["ncert_chapter_link"]:
                        st.markdown(f"**NCERT Chapter:** `{r['ncert_chapter_link']}`")
                    st.markdown(f"💡 *{r['why']}*")
                    if r["link"]:
                        st.markdown(f"[🔗 Open Resource]({r['link']})")

# ── Tab 3: YouTube Videos ─────────────────────
with tab3:
    st.markdown(
        "Clicking the button below will open a **YouTube search** in a new tab "
        "with a query built from your selected subject, level, and topics."
    )
    yt_query_preview = f"NCERT {selected_subject} {selected_level} {search_query} class 11 12".strip()
    st.markdown(f"**Search query:** `{yt_query_preview}`")

    if not search_query:
        st.info("👈 Type a query or select topics to refine the YouTube search.")

    yt_url = build_youtube_url(selected_subject, selected_level, [search_query])
    st.link_button("▶️ Search on YouTube", yt_url, type="primary")

    st.markdown("---")
    st.markdown("**Or jump directly to a level:**")

    level_icons = {"Beginner": "🌱", "Intermediate": "📚", "Advanced": "🔬"}
    yt_levels   = get_levels(selected_subject)
    cols        = st.columns(len(yt_levels))
    for col, lvl in zip(cols, yt_levels):
        with col:
            icon = level_icons.get(lvl, "▶️")
            url  = build_youtube_url(selected_subject, lvl, [search_query])
            st.link_button(f"{icon} {lvl}", url, use_container_width=True)

Overwriting app.py


In [4]:
!pip install pyngrok

In [5]:
import subprocess
import time
from pyngrok import ngrok
import torch

# Verify GPU
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Kill existing streamlit
subprocess.run(["pkill", "-f", "streamlit"], capture_output=True)
time.sleep(2)

# Set ngrok token and kill old tunnels
ngrok.set_auth_token("3A9Zz7jNmMsUvZ0XJkpn32mZ6mK_KJDJPs7MgmMTCd1kuiSo")
ngrok.kill()

# Start Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"]
)

print("Waiting for Streamlit to start...")
time.sleep(20)

# Start tunnel
public_url = ngrok.connect(8501)
print("Open this URL:", public_url)

GPU available: True
Device: Tesla T4
Waiting for Streamlit to start...
Open this URL: NgrokTunnel: "https://nemoricole-mercenarily-wendi.ngrok-free.dev" -> "http://localhost:8501"


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import sqlite3
import numpy as np

DB_PATH = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

# ── 1. Count ────────────────────────────────────────────
cur = conn.cursor()
cur.execute("SELECT COUNT(*) as total FROM pdf_embeddings")
total = cur.fetchone()["total"]
print(f"✅ Total embeddings stored: {total}")

# ── 2. Sample 5 rows ────────────────────────────────────
cur.execute("""
    SELECT pdf_filename, text_hash, created_at,
           LENGTH(embedding) as blob_size
    FROM pdf_embeddings
    ORDER BY created_at DESC
    LIMIT 5
""")
rows = cur.fetchall()
print("\n📋 Latest 5 embeddings:")
print(f"{'Filename':<40} {'Hash':<10} {'Blob (bytes)':<14} {'Created At'}")
print("-" * 90)
for r in rows:
    print(f"{r['pdf_filename']:<40} {r['text_hash'][:8]:<10} {r['blob_size']:<14} {r['created_at']}")

# ── 3. Decode one and check shape ───────────────────────
cur.execute("SELECT pdf_filename, embedding FROM pdf_embeddings LIMIT 1")
row = cur.fetchone()
if row:
    emb = np.frombuffer(row["embedding"], dtype=np.float32)
    print(f"\n🔢 Sample embedding: '{row['pdf_filename']}'")
    print(f"   Shape  : {emb.shape}")
    print(f"   Dtype  : {emb.dtype}")
    print(f"   Min    : {emb.min():.4f}")
    print(f"   Max    : {emb.max():.4f}")
    print(f"   Mean   : {emb.mean():.4f}")
    print(f"   First 5 values: {emb[:5]}")

# ── 4. Check for any nulls / corrupt rows ───────────────
cur.execute("""
    SELECT COUNT(*) as bad
    FROM pdf_embeddings
    WHERE embedding IS NULL OR LENGTH(embedding) < 100
""")
bad = cur.fetchone()["bad"]
print(f"\n{'✅ No corrupt rows found.' if bad == 0 else f'⚠️  {bad} corrupt/empty rows detected!'}")

conn.close()

✅ Total embeddings stored: 84

📋 Latest 5 embeddings:
Filename                                 Hash       Blob (bytes)   Created At
------------------------------------------------------------------------------------------
lepy103.pdf                              a03e9584   1536           2026-02-27 05:18:08
lepy107.pdf                              8c094e71   1536           2026-02-27 05:18:08
lepy1gl.pdf                              798afa09   1536           2026-02-27 05:18:08
lepy102.pdf                              58c1633e   1536           2026-02-27 05:18:08
lepy1ps.pdf                              2eb9ef05   1536           2026-02-27 05:18:08

🔢 Sample embedding: 'keec103.pdf'
   Shape  : (384,)
   Dtype  : float32
   Min    : -0.1250
   Max    : 0.1578
   Mean   : -0.0025
   First 5 values: [-0.08271716 -0.04027534 -0.03839887 -0.04763067  0.03537634]

✅ No corrupt rows found.


In [6]:
import sqlite3

DB_PATH = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row
cur = conn.cursor()

# ── Check all 3 tables ──────────────────────────
cur.execute("SELECT COUNT(*) as n FROM pdf_embeddings")
emb_count = cur.fetchone()["n"]

cur.execute("SELECT COUNT(*) as n FROM topics")
topic_count = cur.fetchone()["n"]

cur.execute("SELECT COUNT(*) as n FROM chapters WHERE is_special = 0")
chapter_count = cur.fetchone()["n"]

print(f"📐 Embeddings : {emb_count}")
print(f"📝 Topics     : {topic_count}")
print(f"📚 Chapters   : {chapter_count}")

if emb_count > 0 and topic_count > 0:
    print("\n✅ DB is fully indexed — PDFs never needed again!")
else:
    missing = []
    if emb_count == 0: missing.append("embeddings")
    if topic_count == 0: missing.append("topics")
    print(f"\n⚠️ Still missing: {', '.join(missing)} — need one more full run")

conn.close()

📐 Embeddings : 84
📝 Topics     : 273
📚 Chapters   : 79

✅ DB is fully indexed — PDFs never needed again!
